In [1]:
from functools import reduce
from matplotlib.lines import Line2D
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap

import pandas as pd
import numpy as np
import xarray as xr
import seaborn as sns
import itertools

import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import matplotlib.gridspec as gridspec
import matplotlib.patheffects as path_effects

In [2]:
# define varaible
interval = 5
list_year_default = list(range(1970, 2101, interval))

start_time_year = 1970
end_time_year = 1980

start_year_ori = list_year_default.index(start_time_year)
end_year_ori = list_year_default.index(end_time_year)

start_time = f"{start_time_year}-01-01"
end_time = f"{end_time_year}-01-01"
frequensi = f"{interval}YS"

start_year_analysis = start_year_ori + 1 # 1975
end_year_analysis = end_year_ori + 1 # 2020

coord_time = pd.date_range(start=start_time, end=end_time, freq=frequensi)
label_start_time_ori = coord_time[start_year_ori]

label_start_time_analysis = coord_time[start_year_analysis]
label_end_time_analysis = coord_time[end_year_ori]

dims = ["time", "latitude", "longitude"]
dims_ngfbfc = ["time", "NGFBFC", "latitude", "longitude"]
lat = 2160
lon = 4320
ocean_label = "ocean"

account_period = len(coord_time) - start_year_ori
analysis_period = account_period - 1

Mg_to_kg = 1e3
Mg_to_tonne = 1
Kg_to_tonne = 1e-3
km2_to_ha = 1e2
ton_to_gigaton = 1e-9
C_to_CO2 = 44/12
N_to_N2O = 44/28
CH4_to_CH4 = 1
CO2_to_CO2eq = 1
tonne_to_tonne = 1
N2O_to_CO2eq = 265
CH4_to_CO2eq = 28
CVKGDMKGC = 0.48 
RCH4CAGW = 5.63 #unit g CH4/ kg C
ratioCH4toC = 0.00563 #(tonne CH4 /tonne C)
CCH4toCH4 = 1
RN2OCAGW = 0.09 #unit g N/ kg C
ratioN2OtoC = 0.00009 #(tonne/tonne)
CNtoN2O = 44/28

std_dtype_str = "<U64"
std_dtype_float = "float32"

columns = ['IMAGE Region Name', 'time', 'NGFBFC']

colors = {'plant based fibres': '#4D869C',
          'non food, luxury, spices': '#7AB2B2',
          'vegetables & fruits': '#CDE8E5',
          'palm oil': '#FC4100',
          'sugar crops': '#FFC55A',
          'tropical roots & tubers': '#8E3E63',
          'temperate roots & tubers': '#D2649A',
          'tropical oil crops': '#03AED2',
          'temperate oil crops': '#68D2E8',
          'soybeans': '#5F6F52',
          'pulses': '#A9B388',
          'temperate cereals': '#FC819E',
          'tropical cereals': '#F7418F',
          'maize': '#FFF455',
          'rice': '#FFEFEF',
          'wheat': '#F7C566',
          "pasture": '#ACE1AF'}

palettes = {'Oceania': '#B3C8CF','Japan':'#E5E483','Korea region':'#D2D180','China region':'#B2B377','Rest of South Asia':'#EF9595',
                'India':'#EFB495','Indonesia region':'#FF8A08','Southeast Asia':'#FFC100','Middle East':'#1B1A55','Central Asia':'#535C91',
                'Russia region':'#9290C3','Rest of Southern Africa':'#C75B7A','South Africa':'#710019','Eastern Africa':'#D43790',
                'Western Africa':'#EC8FD0','Northern Africa':'#F2C5E0', 'Turkey':'#6F4E37','Ukraine region':'#A67B5B','Central Europe':'#ECB176',
                'Western Europe':'#FED8B1', 'Rest of South America':'#254336','Brazil':'#00b2b2','Central America':'#B3E2A7','Mexico':'#003285',
                'USA':'#40A2E3','Canada':'#BBE2EC'}


palette = {
    'maize - Brazil': "#00b2b2", 'soybeans - Brazil': "#00b2b2", 'pulses - Brazil': "#00b2b2",
    'wheat - China region': "#B2B377", 'rice - China region': "#B2B377", 'vegetables & fruits - China region': "#B2B377",
    'non food, luxury, spices - Indonesia region': "#FF8A08", 'palm oil - Indonesia region': "#FF8A08", 'tropical oil crops - Indonesia region': "#FF8A08",
    'tropical roots & tubers - Indonesia region':"#FF8A08", 'pulses - Indonesia region':"#FF8A08", 'sugar crops - Indonesia region':"#FF8A08",
    'soybeans - Rest of South America': "#254336", 'rice - Rest of South Asia': "#EF9595",'wheat - Rest of South Asia': "#EF9595",
    'rice - India': "#EFB495",'rice - Southeast Asia': "#FFC100", 'tropical oil crops - Southeast Asia': "#FFC100", 'non food, luxury, spices - Southeast Asia': "#FFC100",
    'sugar crops - Southeast Asia': "#FFC100",  'tropical cereals - Western Africa': "#EC8FD0", 'tropical roots & tubers - Western Africa': "#EC8FD0",
    'sugar crops - Western Europe': "#FED8B1"
}


markers = {
    'rice - China region': "P", 'rice - Rest of South Asia': "P",
    'rice - India': "P",'rice - Southeast Asia': "P", 'palm oil - Indonesia region': 's', 
    'maize - Brazil': 'o', 'pulses - Brazil': "P", 'soybeans - Brazil': 's',
    'vegetables & fruits - China region': "P", 'wheat - China region': 's', 'wheat - Rest of South Asia': 's',
    'non food, luxury, spices - Indonesia region': 'X', 'non food, luxury, spices - Southeast Asia': 'o', 
    'soybeans - Rest of South America': 'o', 'tropical oil crops - Southeast Asia': 's','tropical oil crops - Indonesia region': "*",
    'tropical cereals - Western Africa': 'X', 'tropical roots & tubers - Western Africa': "P", 'tropical roots & tubers - Indonesia region': "D",
    'sugar crops - Western Europe': 'o', 'sugar crops - Southeast Asia': 'o', 'sugar crops - Indonesia region' : 'o',
    'pulses - Indonesia region':"d"
}

legend_order = [
    'wheat - China region',
    'rice - Southeast Asia','rice - Rest of South Asia','rice - China region',
    'maize - Brazil', 'palm oil - Indonesia region', 'soybeans - Brazil', 
    'tropical oil crops - Indonesia region', 
    'sugar crops - Western Europe','sugar crops - Southeast Asia','sugar crops - Indonesia region', 
    'tropical roots & tubers - Indonesia region',
    'non food, luxury, spices - Indonesia region',
    'tropical cereals - Western Africa',
    'pulses - Indonesia region',]

columns1 = ['Agricultural Transition Emission','Natural Vegetation to Agriculture Emission', 'N2O Land Clearing',
'CO2 Drained Peatland', 'CH4 Drained Peatland','N2O Drained Peatland', 
'N2O Synthetic Fertilizer','N2O Manure Fertilizer',
'CH4 Agriculture Waste Burning', 'N2O Agriculture Waste Burning','N2O Agriculture Residues',
'CH4 Wetland Rice']

colors1 = [
          '#47663B', #Agri Trans
          '#72BF78', #LUC Agri
          '#C2FFC7', #N2O Land clearing
          '#4A628A', #CO2 peat
          '#7AB2D3', #CH4 peat
          '#B9E5E8', #NO2 peat
          '#FA812F', #manure
          '#FAB12F', #synthetic
          '#FFE31A', #burning CH4
          '#FFF100', #burning N2O
          '#FA4032', #agri residue
          '#A04747', #rice
          ]

In [3]:
country_code = pd.read_excel("/INPUT_DATA/ISO-3166-Country-Code_REV.xlsx", engine="openpyxl")
filtred_Ccode = country_code[["ISO Country", "IMAGE Region Name"]]

def matrixMultiply(data1, data2):
    return np.nan_to_num(np.multiply(data1, data2))

def fractionDivision(data1, data2):
    return np.nan_to_num(np.divide(data1, data2))

def save_netcdf(out_file_name, variabel, dims, input_data, coord, time):
    netcdf = xr.Dataset({
        variabel:(dims, input_data)
    }, coords={
        "time": time,
        "latitude": coord.coords["latitude"].to_numpy(),
        "longitude": coord.coords["longitude"].to_numpy()
    })

    netcdf.to_netcdf(f"/OUTPUT_DATA/{out_file_name}.NC", mode='w', format="NETCDF4")
    return netcdf

def save_netcdf_variabel(out_file_name, variabel, input_data, coord, time):
    netcdf = xr.Dataset(
    coords={
        "time": time,
        "latitude": coord.coords["latitude"].to_numpy(),
        "longitude": coord.coords["longitude"].to_numpy(),
    })
    coords = ("time", "latitude", "longitude")
    data_vars = {
        label: (coords, input_data[i]) for i, label in enumerate(variabel)
    }
    netcdf = netcdf.assign(data_vars)

    netcdf.to_netcdf(f"/OUTPUT_DATA/{out_file_name}.NC", mode='w', format="NETCDF4")
    return netcdf

def save_netcdf_ngfbfc(out_file_name, variabel, dims, input_data, input_ngfbfc, coord, time):
    netcdf = xr.Dataset({
        variabel:(dims, input_data)
    },coords={
        "time": time,
        "NGFBFC" : input_ngfbfc,
        "latitude": coord.coords["latitude"].to_numpy(),
        "longitude": coord.coords["longitude"].to_numpy()
    })

    netcdf.to_netcdf(f"/OUTPUT_DATA/{out_file_name}.NC", mode='w', format="NETCDF4")
    return netcdf

def Gprocess(file_name, length, year, input_data1, input_data2, ton):
    Gdata = xr.open_dataset(f"/INPUT_DATA/{file_name}.NC", engine="h5netcdf")
    Gdata = Gdata.isel(time=slice(start_year_analysis, end_year_analysis))

    emission_Gdata_ton_ha = Gdata[f'{file_name}'] * ton / km2_to_ha

    if file_name == "GFERTILIZER":
        GFERTILIZER_manure = Gdata.sel(NFERTSMT=b'manure stable on cropland                         ')
        emission_GFERTILIZER_ton_ha = GFERTILIZER_manure[f"{file_name}"] * Mg_to_tonne / km2_to_ha
        emission_ton = emission_GFERTILIZER_ton_ha * input_data1
        emission_GtTon = emission_ton * (N_to_N2O) * ton_to_gigaton * 5

    else:
        emission_ton = matrixMultiply(emission_Gdata_ton_ha, input_data1)
        emission_GtTon = emission_ton * (N_to_N2O) * ton_to_gigaton * 5

        if file_name == "GECH4RI":
            data_zeros = np.zeros((year, 2160, 4320), dtype="float32")
            for i in range(length):
                for n in range(year):
                    data_zeros[n] = matrixMultiply(emission_GtTon[n], input_data2[i][n])

            return data_zeros

    data_zeros = np.zeros((length, year, 2160, 4320), dtype="float32")
    for i in range(length):
        for n in range(year):
            data_zeros[i][n] = matrixMultiply(emission_GtTon[n], input_data2[i][n])

    return data_zeros

def netcdf_to_excel(out_file_name, input_data, label, columns_name, index_data, column_data, agg):
    df = input_data.isel(time=0).to_dataframe()
    df_pivot = pd.pivot_table(df, values=label, index=index_data, columns=column_data, aggfunc=agg, fill_value=0)
    df_pivot = df_pivot.stack(level=0, future_stack=True)
    df_pivot.columns = pd.to_datetime(df_pivot.columns, format='%d/%m/%Y %H.%M.%S').year

    df_result = df_pivot.reset_index()
    df_result.rename(columns={'level_1': columns_name}, inplace=True)
    region_merge = pd.merge(left=df_result, right=filtred_Ccode, left_on="country_name", right_on="ISO Country")
    region_merge.to_excel(f"/OUTPUT_DATA/{out_file_name}.xlsx", index=False)
    return region_merge

def netcdf_to_excel_peryear(out_file_name, input_data, year, columns_name, input_label, index_data, column_data, agg):
    frames = []
    n = 0
    while n < year:
        df = input_data.isel(time=n).to_dataframe()
        df_table = pd.pivot_table(df, values=input_label, index=index_data, columns=column_data, aggfunc=agg, fill_value=0)
        df_table = df_table.stack(level=0, future_stack=True)

        df_table.columns = pd.to_datetime(df_table.columns, format='%d/%m/%Y %H.%M.%S').year
        df_index = df_table.reset_index()
        df_index.rename(columns={'level_1': columns_name}, inplace=True)
        frames.append(df_index)
        n += 1

    df_result = reduce(lambda left, right: pd.merge(left, right, on=[index_data[0], columns_name]), frames)
    df_result = df_result.replace([np.inf, -np.inf], np.nan)
    df_result = df_result.fillna(0)
    region_merge = pd.merge(left=df_result, right=filtred_Ccode, left_on="country_name", right_on="ISO Country")

    region_merge.to_excel(f"/OUTPUT_DATA/{out_file_name}.xlsx", index=False)
    return region_merge

def gfrac_to_excel(out_file_name, input_data, year, columns_name, input_label, index_data, column_data, agg):
    dataframes = []
    for i, ngfbfc in enumerate(input_label):
        frames = []
        n = 0
        df_ngfbfc = input_data[ngfbfc]
        while n < year:
            df = df_ngfbfc.isel(time=n).to_dataframe()
            df_table = pd.pivot_table(df, index=index_data, columns=column_data, aggfunc=agg, fill_value=0)
            df_table = df_table.stack(level=0, future_stack=True)

            df_table.columns = pd.to_datetime(df_table.columns, format='%d/%m/%Y %H.%M.%S').year
            df_index = df_table.reset_index()
            df_index.rename(columns={'level_1': columns_name}, inplace=True)
            frames.append(df_index)
            n += 1
            
        df_result = reduce(lambda left, right: pd.merge(left, right, on=[index_data[0], columns_name]), frames)
        df_result = df_result.replace([np.inf, -np.inf], np.nan)
        df_result = df_result.fillna(0)
        dataframes.append(df_result)

    merged_df = pd.concat(dataframes, axis=0, ignore_index=True).fillna(0)
    region_merge = pd.merge(left=merged_df, right=filtred_Ccode, left_on="country_name", right_on="ISO Country")
    region_merge.to_excel(f"/OUTPUT_DATA/{out_file_name}.xlsx", index=False)

    return region_merge

def ngfbfc_processing(df):
    try:
        if "type" in df.columns.to_list():
            df = df.rename(columns={"type": "NGFBFC"}) 

        if "NGFBFC" in df.columns:
            df['NGFBFC'] = df['NGFBFC'].str.lower()
            if 'grass' in df['NGFBFC'].unique():
                df['NGFBFC'] = df['NGFBFC'].replace('grass', 'pasture')
            df['NGFBFC'] = df['NGFBFC'].replace("oil & palm fruit", "palm oil")
            df['NGFBFC'] = df['NGFBFC'].replace("other non-food & luxury & spices", "non food, luxury, spices")
            df['NGFBFC'] = df['NGFBFC'].replace('other temperate cereals', 'temperate cereals')
        return df
    except Exception as e:
        print(f"error: {e}")
        return df

def region_processing(df):
    try:
        df['IMAGE Region Name'] = df['IMAGE Region Name'].replace('Kazakhstan region', 'Central Asia').replace('C. Europe', 'Central Europe').replace('E. Africa', 'Eastern Africa').replace('N. Africa', 'Northern Africa')
        df['IMAGE Region Name'] = df['IMAGE Region Name'].replace('Rest C. America', 'Central America').replace('Rest S. Africa', 'Rest of Southern Africa').replace('Rest S. America', 'Rest of South America')
        df['IMAGE Region Name'] = df['IMAGE Region Name'].replace('Rest S. Asia', 'Rest of South Asia').replace('SE. Asia', 'Southeast Asia').replace('Rest S. America', 'Rest of South America')
        df['IMAGE Region Name'] = df['IMAGE Region Name'].replace('W. Africa', 'Western Africa').replace('W. Europe', 'Western Europe').replace('Russia', 'Russia region')
        df['IMAGE Region Name'] = df['IMAGE Region Name'].replace('Indonesia', 'Indonesia region').replace('China', 'China region').replace('Korea', 'Korea region')
        return df
    except Exception as e:
        print(f"error: {e}")
        return df

def custom_mean(x):
  """Calculates the mean of non-zero values."""
  non_zero_values = x[x > 0]
  return non_zero_values.sum() / len(non_zero_values) if len(non_zero_values) > 0 else 0

def get_top_n(df, category_col, value_col, n):
    return df.groupby(category_col).apply(lambda x: x.nlargest(n, value_col)).reset_index(drop=True)

def get_angular_position(NGFBFC_name, NGFBFCs_list):
    """Calculate the angular position of a NGFBFC in the chart"""
    idx = NGFBFCs_list.index(NGFBFC_name)
    return (idx * 360.0 / len(NGFBFCs_list) + 90) % 360

In [4]:
# dataset
glct = xr.open_dataset("/INPUT_DATA/GLCT.NC")
glctdf = pd.read_excel("/INPUT_DATA/GLCT.xlsx", engine="openpyxl", sheet_name="Sheet5", index_col=False)

## Step 1
# S1.AM1.A
label_map = {
    1: 'agri', 2: 'grass', 3: 'carb', 4: 'rfab', 5: 'rftm',
    6: 'biof', 7: 'ice', 8: 'tund', 9: 'tuwd', 10: 'bore',
    11: 'ccfo', 12: 'tmfo', 13: 'tdfo', 14: 'wmfo', 15: 'stepp',
    16: 'dsrt', 17: 'scrb', 18: 'svna', 19: 'trow', 20: 'trof'
}

glct_labels = glct['GLCT'].isel(time=slice(start_year_ori, end_year_analysis)).fillna(0).values.astype(int)
glct_class = np.array([label_map.get(label_id, ocean_label) for label_id in glct_labels.ravel()]).reshape(glct_labels.shape)

glct_NICK_AM1_netcdf = save_netcdf(out_file_name="GLCT_trans", variabel="GLCT_trans", dims=dims, input_data=glct_class, coord=glct, time=pd.date_range(start=label_start_time_ori, end=label_end_time_analysis, freq=frequensi))

glct5mins_trans_AM1 = np.zeros((analysis_period, lat, lon), dtype=std_dtype_str)
for n in range(start_year_ori, analysis_period):
    prev_nicknames_with_to = np.char.add(glct_class[n], '_to_')
    glct5mins_trans_AM1[n] = np.char.add(prev_nicknames_with_to, glct_class[n+1])

glct_trans_AM1_netcdf = save_netcdf(out_file_name="GLCT_trans", variabel="GLCT_trans", dims=dims, input_data=glct5mins_trans_AM1, coord=glct, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

land_use_transitions_postif = dict(glctdf[['Classification_1st', 'Classification_2nd_postif']].values)
land_use_transitions_negatif = dict(glctdf[['Classification_1st', 'Classification_2nd_negatif']].values)
land_use_transitions = dict(glctdf[['trans_name', 'Classification_1st']].values)
glct_class1st_AM1 = np.array([land_use_transitions.get(label_id, 'ocean_to_ocean') for label_id in glct5mins_trans_AM1.ravel()]).reshape(glct5mins_trans_AM1.shape)

trans_glct1st_AM1_netcdf = save_netcdf(out_file_name="GLCT_1st", variabel="GLCT_1st", dims=dims, input_data=glct_class1st_AM1, coord=glct, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

# S1.AM23.A
glct5mins_trans_AM23 = np.zeros((1, lat, lon), dtype=std_dtype_str)
prev_nicknames_with_to = np.char.add(glct_class[start_year_ori], '_to_')
glct5mins_trans_AM23[0] = np.char.add(prev_nicknames_with_to, glct_class[(account_period - start_year_analysis)])

glct_trans_AM23_netcdf = save_netcdf(out_file_name="GLCT_trans", variabel="GLCT_trans", dims=dims, input_data=glct5mins_trans_AM23, coord=glct, time=pd.to_datetime([label_end_time_analysis]))

glct_class1st_AM23 = np.array([land_use_transitions.get(label_id, 'ocean_to_ocean') for label_id in glct5mins_trans_AM23.ravel()]).reshape(glct5mins_trans_AM23.shape)

trans_glct1st_AM23_netcdf = save_netcdf(out_file_name="GLCT_1st", variabel="GLCT_1st", dims=dims, input_data=glct_class1st_AM23, coord=glct, time=pd.to_datetime([label_end_time_analysis]))

# S1.AM1.B
# Country
luh_static = xr.open_dataset("/INPUT_DATA/CCODE_RASTER.nc")
country_code = pd.read_excel("/INPUT_DATA/ISO-3166-Country-Code_REV.xlsx", engine="openpyxl")
gbiomass_5min = xr.open_dataset("/INPUT_DATA/GBIOMASS.NC")

ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['ISO Country'])

ccode_worldwide_int = luh_static['ccode'].to_numpy().astype('int64')
ccode_convert = np.zeros((lat, lon), dtype=std_dtype_str)
cname_dict = {}

for idx, ccode in enumerate(ccode_iso):
    cname_dict[ccode] = cname_iso[idx]

for x in range(lat):
    for y in range(lon):
        if (ccode_worldwide_int[x][y] in cname_dict.keys()):
            ccode_convert[x][y] = cname_dict[ccode_worldwide_int[x][y]]
        else:
            ccode_convert[x][y] = ocean_label

country_coords = xr.Dataset({"country": (["latitude", "longitude"], ccode_convert)},
                         coords={ "longitude": gbiomass_5min.coords["longitude"].to_numpy(),
                                  "latitude": gbiomass_5min.coords["latitude"].to_numpy()})

gbiomass_nbpsum = np.zeros((account_period, lat, lon), dtype=std_dtype_float)
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'long lifetime timber                              ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'short lifetime timber                             ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'charcoal                                          ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'humus                                             ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'litter                                            ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'roots                                             ')

for n in range(account_period):
    gbiomass_nbpsum[n] = gbiomass_5min['GBIOMASS'].isel(time=n).sum(dim='NBP')

gbiomass_biomass_sumdim_netcdf = save_netcdf(out_file_name="gbiomass_biomass_sumdim", variabel="gbiomass_biomass_sumdim", dims=dims, input_data=gbiomass_nbpsum, coord=gbiomass_5min, time=pd.date_range(start=label_start_time_ori, end=label_end_time_analysis, freq=frequensi))

valid_transitions = ['agri_to_biof', 'agri_to_natveg', 'biof_to_natveg', 'grass_to_biof', 'grass_to_natveg', 'natveg_to_agri', 'natveg_to_biof', 'natveg_to_grass']
gbiomass_biomass_sumdim_manip = np.zeros((account_period, lat, lon), dtype=std_dtype_float)
for n in range(account_period): 
    if n <= account_period - 3:
        current_glct = trans_glct1st_AM1_netcdf['GLCT_1st'].isel(time=n).values

        gbiomass_biomass_sumdim_manip[n + 1] = np.where(
            np.isin(current_glct, list(valid_transitions)),
            gbiomass_biomass_sumdim_netcdf['gbiomass_biomass_sumdim'].isel(time=n + 2),
            gbiomass_biomass_sumdim_netcdf['gbiomass_biomass_sumdim'].isel(time=n + 1)
        )

gbiomass_biomass_sumdim_manip_netcdf = save_netcdf(out_file_name="gbiomass_biomass_sumdim_manip", variabel="gbiomass_biomass_sumdim_manip", dims=dims, input_data=gbiomass_biomass_sumdim_manip, coord=gbiomass_biomass_sumdim_netcdf, time=pd.date_range(start=label_start_time_ori, end=label_end_time_analysis, freq=frequensi))

gbiomass_selisih = np.zeros((analysis_period, lat, lon), dtype=std_dtype_float)
for n in range(analysis_period):
    gbiomass_selisih[n] = gbiomass_biomass_sumdim_manip_netcdf['gbiomass_biomass_sumdim_manip'].isel(time=n).to_numpy() - gbiomass_biomass_sumdim_manip_netcdf['gbiomass_biomass_sumdim_manip'].isel(time=n+1).to_numpy()

gbiomass_selisih_AM1_netcdf = save_netcdf(out_file_name="gbiomass_manip_selisih", variabel="gbiomass_manip_selisih", dims=dims, input_data=gbiomass_selisih, coord=gbiomass_5min, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
gbiomass_selisih_AM1_netcdf = gbiomass_selisih_AM1_netcdf.assign_coords(country_name=country_coords['country'])

# S1.AM23.B
gbiomass_selisih = np.zeros((1, lat, lon), dtype=std_dtype_float)
gbiomass_selisih[0] = gbiomass_biomass_sumdim_manip_netcdf['gbiomass_biomass_sumdim_manip'].isel(time=start_year_ori).to_numpy() - gbiomass_biomass_sumdim_manip_netcdf['gbiomass_biomass_sumdim_manip'].isel(time=end_year_ori).to_numpy()

gbiomass_selisih_AM23_netcdf = save_netcdf(out_file_name="gbiomass_manip_selisih", variabel="gbiomass_manip_selisih", dims=dims, input_data=gbiomass_selisih, coord=gbiomass_5min, time=pd.to_datetime([label_end_time_analysis]))

## Step 2
# S2.AM1.A
garea = xr.open_dataset("/INPUT_DATA/GAREA.NC")

LUC_category = ["Agricultural Transition Emission", "Agricultural Transition Sequestration", "LUC Crops Emission", "LUC Crops Sequestration", "Forest Harvest","Forest Growth",
            "Land Abandonment Emission", "Land Abandonment Sequestration", "LUC Rangeland Emission", "LUC Rangeland Sequestration", "LUC Biofuel Emission", "LUC Biofuel Sequestration"]

len_category = len(LUC_category)

garea_ha_7020 = matrixMultiply(garea['GAREA'].to_numpy(), km2_to_ha)
garea_ha_7520 = matrixMultiply(garea['GAREA'].isel(time=slice(start_year_analysis, end_year_analysis)).to_numpy(), km2_to_ha)

gbiomass_tonCha = gbiomass_selisih_AM1_netcdf['gbiomass_manip_selisih'] * Mg_to_tonne / km2_to_ha
emission_C_ton = matrixMultiply(gbiomass_tonCha, garea_ha_7520)
emission_GtTon_CO2 = emission_C_ton * (C_to_CO2) * ton_to_gigaton

emission_CO2_AM1_netcdf = save_netcdf(out_file_name="emission_CO2", variabel="emission_CO2", dims=dims, input_data=emission_GtTon_CO2, coord=garea, time=pd.date_range(start=label_start_time_analysis, end=end_time, freq=frequensi))
ds = xr.merge([trans_glct1st_AM1_netcdf, emission_CO2_AM1_netcdf], compat='no_conflicts', join='outer', fill_value=0)

emission_or_sequestration = ds['emission_CO2'].values > 0
glct_labels = ds['GLCT_1st'].values.astype(str)
glct_class_2nd = np.where(emission_or_sequestration, np.array([land_use_transitions_postif.get(label_id, '') for label_id in glct_labels.ravel()]).reshape(glct_labels.shape),\
                          np.array([land_use_transitions_negatif.get(label_id, '') for label_id in glct_labels.ravel()]).reshape(glct_labels.shape))

trans_glct_2nd_netcdf = save_netcdf(out_file_name="trans_glct", variabel="trans_glct", dims=dims, input_data=glct_class_2nd, coord=garea, time=pd.date_range(start=label_start_time_analysis, end=end_time, freq=frequensi))
ds2 = xr.merge([trans_glct_2nd_netcdf, emission_CO2_AM1_netcdf], compat='no_conflicts', join='outer', fill_value=0)

emission_CO2_occur = np.zeros((len_category, analysis_period, lat, lon), dtype=std_dtype_float)
for i, luc in enumerate(LUC_category):
    for n in range(analysis_period):
        emission_CO2_occur[i][n] = np.where(ds2['trans_glct'].isel(time=n) == luc, ds2['emission_CO2'].isel(time=n), 0)

emission_CO2_LUC_Category_AM1_netcdf = save_netcdf_variabel(out_file_name="emission_CO2_AM1", variabel=LUC_category, input_data=emission_CO2_occur, coord=garea, time=pd.date_range(start=label_start_time_analysis, end=end_time, freq=frequensi))
emission_CO2_LUC_Category_AM1_netcdf = emission_CO2_LUC_Category_AM1_netcdf.assign_coords(country_name=country_coords['country'])
emission_CO2_LUC_Category_AM1_excel = netcdf_to_excel_peryear(out_file_name="emission_CO2_AM1", year=analysis_period, input_data=emission_CO2_LUC_Category_AM1_netcdf, input_label=LUC_category, columns_name="LUC_Category", index_data=['country_name'], column_data=['time'], agg='sum')

# S2.AM23.A
garea_ha_1975 = matrixMultiply(garea['GAREA'].isel(time=1).to_numpy(), km2_to_ha)
gbiomass_tonCha = gbiomass_selisih_AM23_netcdf['gbiomass_manip_selisih'] * Mg_to_tonne / km2_to_ha

emission_C_ton = matrixMultiply(gbiomass_tonCha, garea_ha_1975)
emission_GtTon_CO2 = emission_C_ton * (C_to_CO2) * ton_to_gigaton

emission_CO2_AM23_netcdf = save_netcdf(out_file_name="emission_CO2", variabel="emission_CO2", dims=dims, input_data=emission_GtTon_CO2, coord=garea, time=pd.to_datetime([label_end_time_analysis]))
ds = xr.merge([trans_glct1st_AM23_netcdf, emission_CO2_AM23_netcdf], compat='no_conflicts', join='outer', fill_value=0)

emission_or_sequestration = ds['emission_CO2'].values > 0
glct_labels = ds['GLCT_1st'].values.astype(str)
glct_class_2nd = np.where(emission_or_sequestration, np.array([land_use_transitions_postif.get(label_id, '') for label_id in glct_labels.ravel()]).reshape(glct_labels.shape),\
                          np.array([land_use_transitions_negatif.get(label_id, '') for label_id in glct_labels.ravel()]).reshape(glct_labels.shape))

trans_glct_2nd_netcdf = save_netcdf(out_file_name="trans_glct", variabel="trans_glct", dims=dims, input_data=glct_class_2nd, coord=garea, time=pd.to_datetime([label_end_time_analysis]))
ds2 = xr.merge([trans_glct_2nd_netcdf, emission_CO2_AM23_netcdf], compat='no_conflicts', join='outer', fill_value=0)

emission_CO2_occur = np.zeros((len_category, 1, lat, lon), dtype=std_dtype_float)
for i, luc in enumerate(LUC_category):
    emission_CO2_occur[i][0] = np.where(ds2['trans_glct'].isel(time=0) == luc, ds2['emission_CO2'].isel(time=0), 0)

emission_CO2_AM2_AM3_netcdf = save_netcdf_variabel(out_file_name="emission_CO2_AM2_AM3", variabel=LUC_category, input_data=emission_CO2_occur, coord=garea, time=pd.to_datetime([label_end_time_analysis]))
emission_CO2_AM2_AM3_netcdf = emission_CO2_AM2_AM3_netcdf.assign_coords(country_name=country_coords['country'])
emission_CO2_AM2_AM3_excel = netcdf_to_excel(out_file_name="emission_CO2_AM2_AM3", input_data=emission_CO2_AM2_AM3_netcdf, label=LUC_category, columns_name="LUC_Category", index_data=['country_name'], column_data=['time'], agg='sum')

## Step 3
gfrac5min = xr.open_dataset("/INPUT_DATA/GFRAC.NC")
gfrac_32i = gfrac5min.drop_sel(NGFBFC=b'grass                                             ')
gfrac_32i = gfrac5min.drop_sel(NGFBFC=b'Grains (biofuel)                                  ')
gfrac_32i = gfrac5min.drop_sel(NGFBFC=b'Oil crops (biofuel)                               ')
gfrac_32i = gfrac5min.drop_sel(NGFBFC=b'Sugar cane (biofuel)                              ')
gfrac_32i = gfrac5min.drop_sel(NGFBFC=b'Woody biofuel                                     ')
gfrac_32i = gfrac5min.drop_sel(NGFBFC=b'Non-woody biofuel                                 ')
gfrac_ngfbfc32i = [element.strip() for element in gfrac_32i.coords['NGFBFC'].data.astype('str').tolist()]
gfrac_32i = gfrac_32i.assign_coords(NGFBFC=gfrac_ngfbfc32i)

gfracarea32 = np.zeros((len(gfrac_ngfbfc32i), account_period, lat, lon), dtype="float32")
for i, ngfbfc in enumerate(gfrac_ngfbfc32i):
    for n in range(account_period):
        gfracarea32[i][n] = matrixMultiply(gfrac_32i.isel(time=n).sel(NGFBFC=ngfbfc), garea_ha_7020[n])

gfrac32_netcdf = save_netcdf_variabel(out_file_name="GFRACarea_32_concat", variabel=gfrac_ngfbfc32i, input_data=gfracarea32, coord=garea, time=pd.date_range(start=label_start_time_ori, end=label_end_time_analysis, freq=frequensi))
gfrac32_netcdf = gfrac32_netcdf.assign_coords(country_name=country_coords['country'])
gfrac32_excel = gfrac_to_excel(out_file_name="GFRACarea_32_concat", year=account_period, input_data=gfrac32_netcdf, input_label=gfrac_ngfbfc32i, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

gfrac_ngfbfc_ori = gfrac5min.coords['NGFBFC'].data.tolist()
gfrac_ngfbfc_ir_rf = []
gfrac_ngfbfc_rf = []
gfrac_ngfbfc_ir = []

for i, gfrac_cls in enumerate(gfrac_ngfbfc_ori):
    gfrac_cls_str = gfrac_cls.decode('utf-8') 
    if np.char.replace(gfrac_cls_str, ' ', '') == "grass":
        gfrac_ngfbfc_ir.append(gfrac_cls)
        gfrac_ngfbfc_rf.append(gfrac_cls)
        gfrac_ngfbfc_ir_rf.append(gfrac_cls)
    elif gfrac_cls[0:2] == b'RF':
        gfrac_ngfbfc_rf.append(gfrac_cls)
        gfrac_ngfbfc_ir_rf.append(gfrac_cls[3:])
    elif gfrac_cls[0:2] == b'IR':
        gfrac_ngfbfc_ir.append(gfrac_cls)
        gfrac_ngfbfc_ir_rf.append(gfrac_cls[3:])

gfrac_ngfbfc_ir_rf_list = gfrac_ngfbfc_ir_rf[:17]

gfrac5mins_new = np.zeros((account_period, len(gfrac_ngfbfc_ir_rf_list), lat, lon), dtype=std_dtype_float) #30 mins an
for n in range(account_period):
    gfrac5mins_new[n][0] = gfrac5min["GFRAC"].isel(time=n, NGFBFC=0)
    gfrac5mins_new[n][1] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=1) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=22))
    gfrac5mins_new[n][2] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=2) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=23))
    gfrac5mins_new[n][3] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=3) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=24))
    gfrac5mins_new[n][4] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=4) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=25))
    gfrac5mins_new[n][5] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=5) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=26))
    gfrac5mins_new[n][6] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=6) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=27))
    gfrac5mins_new[n][7] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=7) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=28))
    gfrac5mins_new[n][8] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=8) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=29))
    gfrac5mins_new[n][9] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=9) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=30))
    gfrac5mins_new[n][10] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=10) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=31))
    gfrac5mins_new[n][11] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=11) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=32))
    gfrac5mins_new[n][12] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=12) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=33))
    gfrac5mins_new[n][13] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=13) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=34))
    gfrac5mins_new[n][14] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=14) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=35))
    gfrac5mins_new[n][15] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=15) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=36))
    gfrac5mins_new[n][16] = (gfrac5min["GFRAC"].isel(time=n, NGFBFC=16) + gfrac5min["GFRAC"].isel(time=n, NGFBFC=37))

gfrac_combined_netcdf = save_netcdf_ngfbfc(out_file_name="GFRAC_5_mins_combined_run2", variabel="GFRAC_combined", dims=dims_ngfbfc, input_data=gfrac5mins_new, input_ngfbfc=gfrac_ngfbfc_ir_rf_list, coord=gfrac5min, time=pd.date_range(start=label_start_time_ori, end=label_end_time_analysis, freq=frequensi))

# S3.AM1.A
columns_year_analysis = range(start_time_year+5, end_time_year, interval)

gfrac_ngfbfc = [element.strip() for element in gfrac_combined_netcdf.coords['NGFBFC'].data.astype('str').tolist()]
gfrac_combined_netcdf = gfrac_combined_netcdf.assign_coords(NGFBFC=gfrac_ngfbfc)
len_ngfbfc = len(gfrac_ngfbfc)

gfrac_area_7020 = np.zeros((account_period, len_ngfbfc, lat, lon), dtype=std_dtype_float)
for n in range(account_period):
    for i, crop in enumerate(gfrac_ngfbfc):
        gfrac_area_7020[n][i] = matrixMultiply(gfrac_combined_netcdf['GFRAC_combined'].isel(time=n).sel(NGFBFC=crop).to_numpy(), garea_ha_7020[n])

gfrac_area_7020_netcdf = save_netcdf_ngfbfc(out_file_name="GFRAC_combined", variabel="GFRAC_combined", dims=dims_ngfbfc, input_data=gfrac_area_7020, input_ngfbfc=gfrac_ngfbfc, coord=garea, time=pd.date_range(start=label_start_time_ori, end=label_end_time_analysis, freq=frequensi))
gfrac_area_7020_netcdf = gfrac_area_7020_netcdf.assign_coords(country_name=country_coords['country'])
gfrac_area_7020_excel = gfrac_to_excel(out_file_name="GFRAC_combined", year=account_period, input_data=gfrac_area_7020_netcdf, input_label=gfrac_ngfbfc, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

gfrac_area_7520_netcdf = gfrac_area_7020_netcdf.isel(time=slice(start_year_analysis, end_year_analysis))

prop_crops_GFRAC_AM1 = np.zeros((len_ngfbfc, analysis_period, lat, lon), dtype=std_dtype_float)
for i, crop in enumerate(gfrac_ngfbfc):
    for n in range(analysis_period):
        prop_crops_GFRAC_AM1[i][n] = fractionDivision(gfrac_area_7520_netcdf['GFRAC_combined'].isel(time=n).sel(NGFBFC=crop).to_numpy(),
                                                            gfrac_area_7520_netcdf['GFRAC_combined'].isel(time=n).sum(dim='NGFBFC').to_numpy())

prop_crops_GFRAC_AM1_netcdf = save_netcdf_variabel(out_file_name="prop_crops_GFRAC_AM1", variabel=gfrac_ngfbfc, input_data=prop_crops_GFRAC_AM1, coord=garea, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

luc_crops_prop_crop = np.zeros((len_ngfbfc, analysis_period, lat, lon), dtype=std_dtype_float)
LUCrops_trans = emission_CO2_LUC_Category_AM1_netcdf['LUC Crops Emission'].isel(time=slice(start_year_ori, analysis_period)).to_numpy()
for i, crop in enumerate(gfrac_ngfbfc):
    for n in range(analysis_period):
        luc_crops_prop_crop[i][n] = matrixMultiply(LUCrops_trans[n], prop_crops_GFRAC_AM1[i][n])

luc_crops_prop_crop_AM1_netcdf = save_netcdf_variabel(out_file_name="luc_prop_crops_AM1", variabel=gfrac_ngfbfc, input_data=luc_crops_prop_crop, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
luc_crops_prop_crop_AM1_netcdf = luc_crops_prop_crop_AM1_netcdf.assign_coords(country_name=country_coords['country'])
luc_crops_prop_crop_AM1_excel = netcdf_to_excel_peryear(out_file_name="luc_prop_crops_AM1", year=analysis_period, input_data=luc_crops_prop_crop_AM1_netcdf, input_label=gfrac_ngfbfc, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

luc_crops_prop_crop_AM1_excel.insert(3, "emissions", "LUC_AGRI")
luc_crops_prop_crop_AM1_excel.to_excel(f"/OUTPUT_DATA/luc_prop_crops_AM1.xlsx", index=False)

selisih_gfrac = np.zeros((analysis_period, len_ngfbfc, lat, lon), dtype=std_dtype_float)
for n in range(analysis_period):
    for i, crop in enumerate(gfrac_ngfbfc):
        selisih_gfrac[n][i] = np.where((trans_glct1st_AM1_netcdf['GLCT_1st'].isel(time=n).values == 'agri_to_agri') &
                            (gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=n+1).sel(NGFBFC=crop).values > gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=n).sel(NGFBFC=crop).values),
                            gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=n+1).sel(NGFBFC=crop).values - gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=n).sel(NGFBFC=crop).values, 0)

selisih_gfrac_area_AM1_netcdf = save_netcdf_ngfbfc(out_file_name="selisih_gfrac", variabel="selisih_gfrac", dims=dims_ngfbfc, input_data=selisih_gfrac, input_ngfbfc=gfrac_ngfbfc, coord=garea, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

proporsi_delta = np.zeros((analysis_period, len_ngfbfc, lat, lon), dtype=std_dtype_float)
for n in range(analysis_period):
    for i, crop in enumerate(gfrac_ngfbfc):
        proporsi_delta[n][i] = fractionDivision(selisih_gfrac_area_AM1_netcdf['selisih_gfrac'].isel(time=n).sel(NGFBFC=crop).to_numpy(),
                                                    selisih_gfrac_area_AM1_netcdf['selisih_gfrac'].isel(time=n).sum(dim='NGFBFC').to_numpy())

proporsi_delta_netcdf = save_netcdf_ngfbfc(out_file_name="selisih_gfrac", variabel="selisih_gfrac", dims=dims_ngfbfc, input_data=proporsi_delta, input_ngfbfc=gfrac_ngfbfc, coord=garea, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

agri_to_agri_prop_crop = np.zeros((len_ngfbfc, analysis_period, lat, lon), dtype=std_dtype_float)
agri_trans = emission_CO2_LUC_Category_AM1_netcdf['Agricultural Transition Emission'].isel(time=slice(start_year_ori, analysis_period)).to_numpy()
for i, crop in enumerate(gfrac_ngfbfc):
    for n in range(analysis_period):
        agri_to_agri_prop_crop[i][n] = matrixMultiply(agri_trans[n], proporsi_delta[n][i])

agri_to_agri_prop_crop_AM1_netcdf = save_netcdf_variabel(out_file_name="agri_to_agri_prop_crop_AM1", variabel=gfrac_ngfbfc, input_data=agri_to_agri_prop_crop, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
agri_to_agri_prop_crop_AM1_netcdf = agri_to_agri_prop_crop_AM1_netcdf.assign_coords(country_name=country_coords['country'])
agri_to_agri_prop_crop_AM1_excel = netcdf_to_excel_peryear(out_file_name="agri_to_agri_prop_crop_AM1", year=analysis_period, input_data=agri_to_agri_prop_crop_AM1_netcdf, input_label=gfrac_ngfbfc, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

agri_to_agri_prop_crop_AM1_excel.insert(3, "emissions", "AGRI_to_AGRI")
agri_to_agri_prop_crop_AM1_excel.to_excel(f"/OUTPUT_DATA/agri_to_agri_prop_crop_AM1.xlsx", index=False)

emission_total_AM1 = np.zeros((len_ngfbfc, analysis_period, lat, lon))
for i, crop in enumerate(gfrac_ngfbfc):
    for n in range(analysis_period):
        condition = agri_to_agri_prop_crop[i][n] + luc_crops_prop_crop[i][n]
        emission_total_AM1[i][n] = np.where(np.isfinite(condition), condition, 0)

emission_total_prop_crop_AM1_netcdf = save_netcdf_variabel(out_file_name="emission_total_prop_crop_AM1", variabel=gfrac_ngfbfc, input_data=emission_total_AM1, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
emission_total_prop_crop_AM1_netcdf = emission_total_prop_crop_AM1_netcdf.assign_coords(country_name=country_coords['country'])
emission_total_prop_crop_AM1_excel = netcdf_to_excel_peryear(out_file_name="emission_total_prop_crop_AM1", year=analysis_period, input_data=emission_total_prop_crop_AM1_netcdf, input_label=gfrac_ngfbfc, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

emission_total_prop_crop_AM1_excel.insert(3, "emissions", "emissions_total")
emission_total_prop_crop_AM1_excel.columns[:, columns_year_analysis] = emission_total_prop_crop_AM1_excel.columns[columns_year_analysis].values / interval
emission_total_prop_crop_AM1_excel.to_excel(f"/OUTPUT_DATA/emission_total_prop_crop_AM1.xlsx", index=False)

# S3.AM2.A
prop_crops_GFRAC_AM2 = np.zeros((len_ngfbfc, 1, lat, lon), dtype=std_dtype_float)
for i, crop in enumerate(gfrac_ngfbfc):
    prop_crops_GFRAC_AM2[i][0] = fractionDivision(gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=end_year_ori).sel(NGFBFC=crop).to_numpy(),
                                                            gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=end_year_ori).sum(dim='NGFBFC').to_numpy())

luc_crops_prop_crop = np.zeros((len_ngfbfc, 1, lat, lon), dtype=std_dtype_float)
for i, crop in enumerate(gfrac_ngfbfc):
    luc_crops_prop_crop[i][0] = matrixMultiply(emission_CO2_AM2_AM3_netcdf['LUC Crops Emission'].isel(time=0).to_numpy(), prop_crops_GFRAC_AM2[i][0])

luc_crops_prop_crop_AM2_netcdf = save_netcdf_variabel(out_file_name="luc_crops_prop_crop_AM2", variabel=gfrac_ngfbfc, input_data=luc_crops_prop_crop, coord=gfrac_combined_netcdf, time=pd.to_datetime([label_end_time_analysis]))
luc_crops_prop_crop_AM2_netcdf = luc_crops_prop_crop_AM2_netcdf.assign_coords(country_name=country_coords['country'])
luc_crops_prop_crop_AM2_excel = netcdf_to_excel(out_file_name="luc_crops_prop_crop_AM2", input_data=luc_crops_prop_crop_AM2_netcdf, label=gfrac_ngfbfc, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

selisih_gfrac = np.zeros((1, len_ngfbfc, lat, lon), dtype=std_dtype_float)
for i, crop in enumerate(gfrac_ngfbfc):
    selisih_gfrac[0][i] = np.where(trans_glct1st_AM23_netcdf['GLCT_1st'].isel(time=0).values == 'agri_to_agri',
                                                np.where(gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=end_year_ori).sel(NGFBFC=crop).values > gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=start_year_ori).sel(NGFBFC=crop).values,
                                                      gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=end_year_ori).sel(NGFBFC=crop).values - gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=start_year_ori).sel(NGFBFC=crop).values, 0), 0)

selisih_gfrac_area_7020_netcdf = save_netcdf_ngfbfc(out_file_name="selisih_gfrac", variabel="selisih_gfrac", dims=dims_ngfbfc, input_data=selisih_gfrac, input_ngfbfc=gfrac_ngfbfc, coord=garea, time=pd.to_datetime([label_end_time_analysis]))

proporsi_delta = np.zeros((1, len_ngfbfc, lat, lon), dtype=std_dtype_float)
for i, crop in enumerate(gfrac_ngfbfc):
        proporsi_delta[0][i] = fractionDivision(selisih_gfrac_area_7020_netcdf['selisih_gfrac'].isel(time=0).sel(NGFBFC=crop).to_numpy(),
                                                    selisih_gfrac_area_7020_netcdf['selisih_gfrac'].isel(time=0).sum(dim='NGFBFC').to_numpy())

agri_to_agri_prop_crop = np.zeros((len_ngfbfc, 1, lat, lon), dtype=std_dtype_float)
agri_trans = emission_CO2_AM2_AM3_netcdf['Agricultural Transition Emission'].to_numpy()
for i, crop in enumerate(gfrac_ngfbfc):
        agri_to_agri_prop_crop[i][0] = matrixMultiply(agri_trans[0], proporsi_delta[0][i])

agri_to_agri_prop_crop_AM2_netcdf = save_netcdf_variabel(out_file_name="agri_to_agri_prop_crop_AM2", variabel=gfrac_ngfbfc, input_data=agri_to_agri_prop_crop, coord=gfrac_combined_netcdf, time=pd.to_datetime([label_end_time_analysis]))
agri_to_agri_prop_crop_AM2_netcdf = agri_to_agri_prop_crop_AM2_netcdf.assign_coords(country_name=country_coords['country'])
agri_to_agri_prop_crop_AM2_excel = netcdf_to_excel(out_file_name="agri_to_agri_prop_crop_AM2", input_data=agri_to_agri_prop_crop_AM2_netcdf, label=gfrac_ngfbfc, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

emission_total_AM2 = np.zeros((len_ngfbfc, 1, lat, lon))
for i, crop in enumerate(gfrac_ngfbfc):
    condition = agri_to_agri_prop_crop[i][0] + luc_crops_prop_crop[i][0]
    emission_total_AM2[i][0] = np.where(np.isfinite(condition), condition, 0)

emission_total_prop_crop_AM2_netcdf = save_netcdf_variabel(out_file_name="emission_total_prop_crop_AM2", variabel=gfrac_ngfbfc, input_data=emission_total_AM2, coord=gfrac_combined_netcdf, time=pd.to_datetime([label_end_time_analysis]))
emission_total_prop_crop_AM2_netcdf = emission_total_prop_crop_AM2_netcdf.assign_coords(country_name=country_coords['country'])
emission_total_prop_crop_AM2_excel = netcdf_to_excel(out_file_name="emission_total_prop_crop_AM2", input_data=emission_total_prop_crop_AM2_netcdf, label=gfrac_ngfbfc, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

# S3.AM3.A
akumulasi = np.zeros((1, len_ngfbfc, lat, lon))
for n in range(account_period):
    for i, crop in enumerate(gfrac_ngfbfc):
        condition = np.where(gfrac_combined_netcdf['GFRAC_combined'].isel(time=n).sel(NGFBFC=crop).to_numpy() > 0, gfrac_combined_netcdf['GFRAC_combined'].isel(time=n).sel(NGFBFC=crop).to_numpy(), 0)
        akumulasi[0][i] += np.nan_to_num(condition)
        akumulasi[0][i] = np.nan_to_num(akumulasi[0][i])

akumulasi_netcdf = save_netcdf_ngfbfc(out_file_name="akumulasi", variabel="akumulasi", dims=dims_ngfbfc, input_data=akumulasi, input_ngfbfc=gfrac_ngfbfc, coord=gfrac_combined_netcdf, time=pd.to_datetime([label_end_time_analysis]))

glct_array = np.where(glct_NICK_AM1_netcdf['GLCT_trans'].to_numpy() == "agri", 1, 0)
glct_numpy = np.zeros((1, lat, lon))
for n in range(analysis_period):
    glct_numpy[0] += np.nan_to_num(glct_array[n])

proporsi_OY = np.zeros((1, len_ngfbfc, lat, lon))
for i, crop in enumerate(gfrac_ngfbfc):
    condition = fractionDivision(akumulasi[0][i], glct_numpy[0])
    proporsi_OY[0][i] = np.nan_to_num(condition)

proporsi_AM3_netcdf = save_netcdf_ngfbfc(out_file_name="proporsi_AM3_OY_coord", variabel="proporsi_OY", dims=dims_ngfbfc, input_data=proporsi_OY, input_ngfbfc=gfrac_ngfbfc, coord=gfrac_combined_netcdf, time=pd.to_datetime([label_end_time_analysis]))
proporsi_AM3_netcdf = proporsi_AM3_netcdf.assign_coords(country_name=country_coords['country'])

proporsi_OY = proporsi_OY.reshape((len_ngfbfc, 1, lat, lon))
proporsi2_AM3_netcdf = save_netcdf_variabel(out_file_name="proporsi_AM3_OY_variabel", variabel=gfrac_ngfbfc, input_data=proporsi_OY, coord=gfrac_combined_netcdf, time=pd.to_datetime([label_end_time_analysis]))

luc_prop_crop = np.zeros((len_ngfbfc, 1, lat, lon), dtype=std_dtype_float)
for i, crop in enumerate(gfrac_ngfbfc):
    luc_prop_crop[i][0] = matrixMultiply(emission_CO2_AM2_AM3_netcdf['LUC Crops Emission'].isel(time=0).to_numpy(), proporsi_OY[i][0])

luc_crops_prop_crop_AM3_netcdf = save_netcdf_variabel(out_file_name="luc_crops_prop_crop_AM3", variabel=gfrac_ngfbfc, input_data=luc_prop_crop, coord=gfrac_combined_netcdf, time=pd.to_datetime([label_end_time_analysis]))
luc_crops_prop_crop_AM3_netcdf = luc_crops_prop_crop_AM3_netcdf.assign_coords(country_name=country_coords['country'])
luc_crops_prop_crop_AM3_excel = netcdf_to_excel(out_file_name="luc_crops_prop_crop_AM3", input_data=luc_crops_prop_crop_AM3_netcdf, label=gfrac_ngfbfc, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

agri_to_agri_prop_crop = np.zeros((len_ngfbfc, 1, lat, lon))
for i, crop in enumerate(gfrac_ngfbfc):
    agri_to_agri_prop_crop[i][0] = matrixMultiply(emission_CO2_AM2_AM3_netcdf['Agricultural Transition Emission'].isel(time=0).to_numpy(), proporsi_OY[i][0])

agri_to_agri_prop_crop_AM3_netcdf = save_netcdf_variabel(out_file_name="agri_to_agri_prop_crop_AM3", variabel=gfrac_ngfbfc, input_data=agri_to_agri_prop_crop, coord=gfrac_combined_netcdf, time=pd.to_datetime([label_end_time_analysis]))
agri_to_agri_prop_crop_AM3_netcdf = agri_to_agri_prop_crop_AM3_netcdf.assign_coords(country_name=country_coords['country'])
agri_to_agri_prop_crop_AM3_excel = netcdf_to_excel(out_file_name="agri_to_agri_prop_crop_AM3", input_data=agri_to_agri_prop_crop_AM3_netcdf, label=gfrac_ngfbfc, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

emission_total_AM3 = np.zeros((len_ngfbfc, 1, lat, lon))
for i, crop in enumerate(gfrac_ngfbfc):
    condition = agri_to_agri_prop_crop[i][0] + luc_prop_crop[i][0]
    emission_total_AM3[i][0] = np.where(np.isfinite(condition), condition, 0)

emission_total_prop_crop_AM3_netcdf = save_netcdf_variabel(out_file_name="emission_total_prop_crop_AM3", variabel=gfrac_ngfbfc, input_data=emission_total_AM3, coord=gfrac_combined_netcdf, time=pd.to_datetime([label_end_time_analysis]))
emission_total_prop_crop_AM3_netcdf = emission_total_prop_crop_AM3_netcdf.assign_coords(country_name=country_coords['country'])
emission_total_prop_crop_AM3_excel = netcdf_to_excel(out_file_name="emission_total_prop_crop_AM3", input_data=emission_total_prop_crop_AM3_netcdf, label=gfrac_ngfbfc, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')


In [ ]:
## Step 4
# S4.1A
ngfbfc_wt_grass = [element for element in gfrac_ngfbfc if element != 'grass']
len_wt_grass = len(ngfbfc_wt_grass)
list_gdata = {"GEN2OLC_N20": [len_ngfbfc, Kg_to_tonne], "GFERTILIZER_N20": [len_wt_grass, Mg_to_tonne], "GECH4RI_CH4": [1, Kg_to_tonne]}

for key, value in list_gdata.items():
    filename = key.split('_')[0]
    gas_type = key.split('_')[1]

    if filename == "GFERTILIZER":
        gdata = Gprocess(file_name=filename, length=value[0], year=analysis_period, input_data1=garea_ha_7520, input_data2=prop_crops_GFRAC_AM1, ton=value[1])
        Gdata_crop_netcdf = save_netcdf_variabel(out_file_name=f"{filename}_crop_{gas_type}", variabel=ngfbfc_wt_grass, input_data=gdata, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

        Gdata_crop_netcdf = Gdata_crop_netcdf.assign_coords(country_name=country_coords["country"])
        Gdata_crop_excel = netcdf_to_excel_peryear(out_file_name=f"{filename}_crop_{gas_type}", input_data=Gdata_crop_netcdf, year=analysis_period, columns_name="NGFBFC", input_label=ngfbfc_wt_grass, index_data=["country_name"], column_data=['time'], agg="sum")

        Gdata_crop_excel.insert(3, "emissions", f"{filename}_crop_{gas_type}")
        Gdata_crop_excel.columns[:, columns_year_analysis] = (Gdata_crop_excel.columns[columns_year_analysis].values * N2O_to_CO2eq * 0.002) / interval
        Gdata_crop_excel.to_excel(f"/OUTPUT_DATA/{filename}_crop_{gas_type}.xlsx", index=False)
    
    elif filename == "GECH4RI":
        gdata = Gprocess(file_name=filename, length=value[0], year=analysis_period, input_data1=garea_ha_7520, input_data2=prop_crops_GFRAC_AM1, ton=value[1])
        Gdata_crop_netcdf = save_netcdf(out_file_name=f"{filename}_crop_{gas_type}", variabel="rice", dims=dims, input_data=gdata, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

        Gdata_crop_netcdf = Gdata_crop_netcdf.assign_coords(country_name=country_coords["country"])
        Gdata_crop_excel = netcdf_to_excel_peryear(out_file_name=f"{filename}_crop_{gas_type}", input_data=Gdata_crop_netcdf, year=analysis_period, columns_name="NGFBFC", input_label="rice", index_data=["country_name"], column_data=['time'], agg="sum")

        Gdata_crop_excel.insert(3, "emissions", f"{filename}_crop_{gas_type}")
        Gdata_crop_excel.columns[:, columns_year_analysis] = (Gdata_crop_excel.columns[columns_year_analysis].values * CH4_to_CO2eq) / interval        
        Gdata_crop_excel.to_excel(f"/OUTPUT_DATA/{filename}_crop_{gas_type}.xlsx", index=False)

    else:
        gdata = Gprocess(file_name=filename, length=value[0], year=analysis_period, input_data1=garea_ha_7520, input_data2=prop_crops_GFRAC_AM1, ton=value[1])
        Gdata_crop_netcdf = save_netcdf_variabel(out_file_name=f"{filename}_crop_{gas_type}",  variabel=gfrac_ngfbfc, input_data=gdata, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

        Gdata_crop_netcdf = Gdata_crop_netcdf.assign_coords(country_name=country_coords["country"])
        Gdata_crop_excel = netcdf_to_excel_peryear(out_file_name=f"{filename}_crop_{gas_type}", input_data=Gdata_crop_netcdf, year=analysis_period, columns_name="NGFBFC", input_label=gfrac_ngfbfc, index_data=["country_name"], column_data=['time'], agg="sum")

        Gdata_crop_excel.insert(3, "emissions", f"{filename}_crop_{gas_type}")
        Gdata_crop_excel.columns[:, columns_year_analysis] = (Gdata_crop_excel.columns[columns_year_analysis].values * N2O_to_CO2eq) / interval
        Gdata_crop_excel.to_excel(f"/OUTPUT_DATA/{filename}_crop_{gas_type}.xlsx", index=False)

GFERTILIZERSYN = xr.open_dataset("/INPUT_DATA/GFERTILIZERSYN.NC")
GFERTILIZERSYN = GFERTILIZERSYN.drop_sel(NFBCT=b'Grains (biofuel)                                  ')
GFERTILIZERSYN = GFERTILIZERSYN.drop_sel(NFBCT=b'Oil crops (biofuel)                               ')
GFERTILIZERSYN = GFERTILIZERSYN.drop_sel(NFBCT=b'Sugar cane (biofuel)                              ')
GFERTILIZERSYN = GFERTILIZERSYN.drop_sel(NFBCT=b'Woody biofuel                                     ')
GFERTILIZERSYN = GFERTILIZERSYN.drop_sel(NFBCT=b'Non-woody biofuel                                 ')
GFERTILIZERSYN = GFERTILIZERSYN.drop_sel(NFBCT=b'total                                             ')
GFERTILIZERSYN = GFERTILIZERSYN.isel(time=slice(start_year_analysis, end_year_analysis))
GFERTILIZERSYN_ton_ha = GFERTILIZERSYN['GFERTILIZERSYN'] * Mg_to_tonne / km2_to_ha

GFERTILIZERSYN_gfrac_area = np.zeros((len(ngfbfc_wt_grass), analysis_period, lat, lon), dtype=std_dtype_float)
for i, crop in enumerate(ngfbfc_wt_grass):
    for n in range(analysis_period):
        GFERTILIZERSYN_gfrac_area[i][n] = matrixMultiply(GFERTILIZERSYN_ton_ha.isel(time=n, NFBCT=i).to_numpy(), gfrac_area_7520_netcdf['GFRAC_combined'].isel(time=n, NGFBFC=i).to_numpy())
        
emission_GFERTILIZERSYN_GtTon = GFERTILIZERSYN_gfrac_area * (N_to_N2O) * ton_to_gigaton * interval

emission_GFERTILIZERSYN_GtTon_netcdf = save_netcdf_variabel(out_file_name="GFERTILIZER_SYNTHETIC_crop_N20", variabel=ngfbfc_wt_grass, input_data=emission_GFERTILIZERSYN_GtTon, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
emission_GFERTILIZERSYN_GtTon_netcdf = emission_GFERTILIZERSYN_GtTon_netcdf.assign_coords(country_name=country_coords["country"])
emission_GFERTILIZERSYN_GtTon_excel = netcdf_to_excel_peryear(out_file_name="GFERTILIZER_SYNTHETIC_crop_N20", input_data=emission_GFERTILIZERSYN_GtTon_netcdf, year=analysis_period, columns_name="NGFBFC", input_label=ngfbfc_wt_grass, index_data=["country_name"], column_data=['time'], agg="sum")

emission_GFERTILIZERSYN_GtTon_excel.insert(3, "emissions", "GFERTILIZER_SYNTHETIC")
emission_GFERTILIZERSYN_GtTon_excel.columns[:, columns_year_analysis] = (emission_GFERTILIZERSYN_GtTon_excel.columns[columns_year_analysis].values * N2O_to_CO2eq * 0.01) / interval
emission_GFERTILIZERSYN_GtTon_excel.to_excel(f"/OUTPUT_DATA/GFERTILIZER_SYNTHETIC_crop_N20.xlsx", index=False)

# S4.2A
GABOVERES = xr.open_dataset("/INPUT_DATA/GABOVERES.NC", engine='netcdf4')
AGWBUR = pd.read_excel("/INPUT_DATA/AGWBUR.xlsm")
GREG = xr.open_dataset("/INPUT_DATA/GREG.nc", engine='netcdf4')
GEN2ORE = xr.open_dataset("/INPUT_DATA/GEN2ORE.NC", engine="netcdf4")

GABOVERES_ngfbfc_ori = GABOVERES.coords['NGFBFC'].data.tolist()

GABOVERES_ngfbfc_ir_rf = []
for i, GABOVERES_cls in enumerate(GABOVERES_ngfbfc_ori):
    GABOVERES_cls_str = GABOVERES_cls.decode('utf-8')
    if np.char.replace(GABOVERES_cls_str, ' ', '') == "grass":
        GABOVERES_ngfbfc_ir_rf.append(GABOVERES_cls)
    elif GABOVERES_cls_str.startswith('RF'):
        GABOVERES_ngfbfc_ir_rf.append(GABOVERES_cls[3:])
    elif GABOVERES_cls_str.startswith('IR'):
        GABOVERES_ngfbfc_ir_rf.append(GABOVERES_cls[3:])

GABOVERES_ngfbfc_ir_rf_list = GABOVERES_ngfbfc_ir_rf[:17]

GABOVERES_new = np.zeros((account_period, len(GABOVERES_ngfbfc_ir_rf_list), lat, lon), dtype=std_dtype_float)
for n in range(account_period):
    GABOVERES_new[n][0] = GABOVERES["GABOVERES"].isel(time=n, NGFBFC=0)
    GABOVERES_new[n][1] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=1) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=22))
    GABOVERES_new[n][2] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=2) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=23))
    GABOVERES_new[n][3] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=3) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=24))
    GABOVERES_new[n][4] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=4) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=25))
    GABOVERES_new[n][5] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=5) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=26))
    GABOVERES_new[n][6] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=6) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=27))
    GABOVERES_new[n][7] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=7) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=28))
    GABOVERES_new[n][8] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=8) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=29))
    GABOVERES_new[n][9] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=9) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=30))
    GABOVERES_new[n][10] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=10) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=31))
    GABOVERES_new[n][11] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=11) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=32))
    GABOVERES_new[n][12] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=12) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=33))
    GABOVERES_new[n][13] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=13) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=34))
    GABOVERES_new[n][14] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=14) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=35))
    GABOVERES_new[n][15] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=15) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=36))
    GABOVERES_new[n][16] = (GABOVERES["GABOVERES"].isel(time=n, NGFBFC=16) + GABOVERES["GABOVERES"].isel(time=n, NGFBFC=37))

GABOVERES_combined_netcdf = save_netcdf_ngfbfc(out_file_name="GABOVERES_combined", variabel="GABOVERES_combined", dims=dims_ngfbfc, input_data=GABOVERES_new, input_ngfbfc=GABOVERES_ngfbfc_ir_rf_list, coord=GABOVERES, time=pd.date_range(start=label_start_time_ori, end=label_end_time_analysis, freq=frequensi))

burnresLocc = np.zeros((account_period, len(GABOVERES_ngfbfc_ir_rf_list), lat, lon), dtype=std_dtype_float)
for n, year in enumerate(range(start_time_year, end_time_year, interval)):
    for i, crops in enumerate(gfrac_ngfbfc_ir_rf_list):
        for j in range(1, 27):
            mask = np.where(GREG['GREG'].isel(time=0).to_numpy() == j)
            burnresLocc[n][i][mask] = matrixMultiply(GABOVERES_combined_netcdf['GABOVERES_combined'].isel(time=n, NGFBFC=i).to_numpy()[mask], AGWBUR[AGWBUR['year'] == year][j].values)

burnresLocc_netcdf = save_netcdf_ngfbfc(out_file_name="burnresLocc", variabel="burnresLocc", dims=dims_ngfbfc, input_data=burnresLocc, input_ngfbfc=GABOVERES_ngfbfc_ir_rf_list, coord=GABOVERES, time=pd.date_range(start=label_start_time_ori, end=label_end_time_analysis, freq=frequensi))

burnresLocc_ngfbfc = [element.strip() for element in burnresLocc_netcdf.coords['NGFBFC'].data.astype('str').tolist()]
eco2abLoc_ton_C_ha = ((burnresLocc_netcdf['burnresLocc'].isel(time=slice(start_year_analysis, end_year_analysis)) * CVKGDMKGC * Mg_to_tonne) / km2_to_ha)

eco2abLoc_ton_C = np.zeros((analysis_period, len(burnresLocc_ngfbfc), lat, lon), dtype=std_dtype_float)
for n in range(analysis_period):
    for i in range(len(burnresLocc_ngfbfc)):
        eco2abLoc_ton_C[n][i] = matrixMultiply(eco2abLoc_ton_C_ha.isel(time=n, NGFBFC=i), garea_ha_7020[n])

eco2abLoc_ton_C_netcdf = save_netcdf_ngfbfc(out_file_name="eco2abLoc_ton_C", variabel="eco2abLoc_ton_C", dims=dims_ngfbfc, input_data=eco2abLoc_ton_C, input_ngfbfc=burnresLocc_ngfbfc, coord=GABOVERES, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

geCH4ab = eco2abLoc_ton_C_netcdf['eco2abLoc_ton_C'].to_numpy() * ratioCH4toC * CCH4toCH4
geCH4ab_GtTon = (geCH4ab * ton_to_gigaton * interval)
geCH4ab_GtTon = geCH4ab_GtTon.reshape((len(burnresLocc_ngfbfc), analysis_period, lat, lon))

geCH4ab_GtTon_netcdf = save_netcdf_variabel(out_file_name="geCH4ab_crop_CH4", variabel=burnresLocc_ngfbfc, input_data=geCH4ab_GtTon, coord=GABOVERES, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
geCH4ab_GtTon_netcdf = geCH4ab_GtTon_netcdf.assign_coords(country_name=country_coords["country"])
geCH4ab_GtTon_excel = netcdf_to_excel_peryear(out_file_name="geCH4ab_crop_CH4", input_data=geCH4ab_GtTon_netcdf, year=analysis_period, columns_name="NGFBFC", input_label=burnresLocc_ngfbfc, index_data=["country_name"], column_data=['time'], agg="sum")

geCH4ab_GtTon_excel = pd.merge(left=geCH4ab_GtTon_excel, right=country_code, left_on="country_name", right_on="ISO Country")
geCH4ab_GtTon_excel = geCH4ab_GtTon_excel[['IMAGE Region Name', 'time', 'NGFBFC']]
geCH4ab_GtTon_excel = geCH4ab_GtTon_excel.groupby(['IMAGE Region Name', 'time', 'NGFBFC']).sum()
geCH4ab_GtTon_excel = geCH4ab_GtTon_excel.reset_index()

geCH4ab_GtTon_excel.to_excel("/OUTPUT_DATA/geCH4ab_crop_CH4.xlsx", index=False)

geN2Oab = eco2abLoc_ton_C_netcdf['eco2abLoc_ton_C'].to_numpy() * ratioN2OtoC * CNtoN2O
geN2Oab_GtTon = (geN2Oab * ton_to_gigaton * interval)
geN2Oab_GtTon = geN2Oab_GtTon.reshape((len(burnresLocc_ngfbfc), analysis_period, lat, lon))

GgeN2Oab_GtTon_netcdf = save_netcdf_variabel(out_file_name="geN2Oab_crop_N2O", variabel=burnresLocc_ngfbfc, input_data=geN2Oab_GtTon, coord=GABOVERES, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
GgeN2Oab_GtTon_netcdf = GgeN2Oab_GtTon_netcdf.assign_coords(country_name=country_coords["country"])
GgeN2Oab_GtTon_excel = netcdf_to_excel_peryear(out_file_name="geN2Oab_crop_N2O", input_data=GgeN2Oab_GtTon_netcdf, year=analysis_period, columns_name="NGFBFC", input_label=burnresLocc_ngfbfc, index_data=["country_name"], column_data=['time'], agg="sum")

GgeN2Oab_GtTon_excel = pd.merge(left=GgeN2Oab_GtTon_excel, right=country_code, left_on="country_name", right_on="ISO Country")
GgeN2Oab_GtTon_excel = GgeN2Oab_GtTon_excel[['IMAGE Region Name', 'time', 'NGFBFC']]
GgeN2Oab_GtTon_excel = GgeN2Oab_GtTon_excel.groupby(['IMAGE Region Name', 'time', 'NGFBFC']).sum()
GgeN2Oab_GtTon_excel = GgeN2Oab_GtTon_excel.reset_index()

GgeN2Oab_GtTon_excel.to_excel("/OUTPUT_DATA/geN2Oab_crop_N2O.xlsx", index=False)

GABOVERES_netcdf = GABOVERES_combined_netcdf.isel(time=slice(start_year_analysis, end_year_analysis))
prop_crops_GABOVERES = np.zeros((len(GABOVERES_ngfbfc_ir_rf_list), analysis_period, lat, lon), dtype=std_dtype_float)
for i, crop in enumerate(GABOVERES_ngfbfc_ir_rf_list):
    for n in range(analysis_period):
        prop_crops_GABOVERES[i][n] = fractionDivision(GABOVERES_netcdf['GABOVERES_combined'].isel(NGFBFC=i).isel(time=n).to_numpy(), 
                                                            GABOVERES_netcdf['GABOVERES_combined'].isel(time=n).sum(dim='NGFBFC').to_numpy())
        
GEN2ORE = GEN2ORE.isel(time=slice(start_year_analysis, end_year_analysis))
emission_GEN2ORE_ton_ha = GEN2ORE['GEN2ORE'] * Kg_to_tonne / km2_to_ha
emission_ton = matrixMultiply(emission_GEN2ORE_ton_ha, garea_ha_7520)
emission_GtTon = emission_ton * (N_to_N2O) * ton_to_gigaton * interval

emission_GtTon_netcdf = save_netcdf_ngfbfc(out_file_name="GEN2ORE", variabel="GEN2ORE", dims=dims_ngfbfc, input_data=emission_GtTon, input_ngfbfc=burnresLocc_ngfbfc, coord=gfrac5min, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

GEN2ORE_crop = np.zeros((len(GABOVERES_ngfbfc_ir_rf_list), analysis_period, lat, lon), dtype=std_dtype_float)
for i, crop in enumerate(burnresLocc_ngfbfc):
    for n in range(analysis_period):
        GEN2ORE_crop[i][n] = matrixMultiply(emission_GtTon_netcdf['GEN2ORE'].isel(time=n).to_numpy(), prop_crops_GABOVERES[i][n])

GEN2ORE_crop_netcdf = save_netcdf_variabel(out_file_name="GEN2ORE_crop_v2", variabel=burnresLocc_ngfbfc, input_data=GEN2ORE_crop, coord=gfrac5min, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
GEN2ORE_crop_netcdf = GEN2ORE_crop_netcdf.assign_coords(country_name=country_coords["country"])
GEN2ORE_crop_excel = netcdf_to_excel_peryear(out_file_name="GEN2ORE_crop_v2", input_data=GEN2ORE_crop_netcdf, year=analysis_period, columns_name="NGFBFC", input_label=burnresLocc_ngfbfc, index_data=["country_name"], column_data=['time'], agg="sum")

GEN2ORE_crop_excel = pd.merge(left=GEN2ORE_crop_excel, right=country_code, left_on="country_name", right_on="ISO Country")
GEN2ORE_crop_excel = GEN2ORE_crop_excel[['IMAGE Region Name', 'time', 'NGFBFC']]
GEN2ORE_crop_excel = GEN2ORE_crop_excel.groupby(['IMAGE Region Name', 'time', 'NGFBFC']).sum()
GEN2ORE_crop_excel = GEN2ORE_crop_excel.reset_index()

GEN2ORE_crop_excel.to_excel("/OUTPUT_DATA/GEN2ORE_crop_v2.xlsx", index=False)

# S4.1B
GPEATLAND = xr.open_dataset("/INPUT_DATA/GPEATLAND.NC", engine="netcdf4")
GPEATLANDSFRAC = xr.open_dataset("/INPUT_DATA/GPEATLANDSFRAC.NC", engine="netcdf4")

peatland_fraction_total_on_crop = np.zeros((account_period, lat, lon), dtype=std_dtype_float)
for n in range(account_period):
    if n < analysis_period:
        peatland_fraction_total_on_crop[n+1] = np.where(
            (GPEATLAND['GPEATLAND'].isel(time=n) == 1) & (GPEATLAND['GPEATLAND'].isel(time=n+1) == 2),
            matrixMultiply(GPEATLANDSFRAC['GPEATLANDSFRAC'].isel(time=0).values, garea_ha_7020[n+1]),
            0
        )

peatland_area_prop_crop = np.zeros((account_period, len_ngfbfc, lat, lon), dtype=std_dtype_float)
for n in range(account_period):
    for i in range(len_ngfbfc):
        peatland_area_prop_crop[n][i] = matrixMultiply(gfrac_area_7020_netcdf['GFRAC_combined'].isel(time=n).isel(NGFBFC=i), peatland_fraction_total_on_crop[n])

peatland_area_prop_crop_netcdf = save_netcdf_ngfbfc(out_file_name="peatland_area_prop_crops", variabel="peatland_area_prop_crops", dims=dims_ngfbfc, input_data=peatland_area_prop_crop, input_ngfbfc=gfrac_ngfbfc, coord=gfrac_combined_netcdf, time=pd.date_range(start=start_time, end=end_time, freq=frequensi))

# S4.2B
GNLCT = xr.open_dataset("/INPUT_DATA/GNLCT.NC", engine="netcdf4")

sheet_list = ['CO2', 'CH4', 'N2O', 'DOC']
peatland_columns = ['Boreal', 'Temperate', 'Tropical']
boreal_range = range(7,12)
temperate_range = range(12,16)
tropical_range = range(16,21)

ef_zeros = np.zeros((analysis_period, len_ngfbfc, lat, lon), dtype='float32')
for sheet in sheet_list:
    peatland_df = pd.read_excel("/INPUT_DATA/EF Peatland.xlsx", sheet_name=sheet)
    peatland_df[peatland_columns] = peatland_df[peatland_columns].replace(',', '.', regex=True).astype(float)

    for n in range(analysis_period):
        gnlct_values = GNLCT['GNLCT'].isel(time=n)
        for i, ngfbfc in enumerate(peatland_df['NGFBFC'].to_list()):

            if np.any(np.isin(gnlct_values, boreal_range)):
                ef_zeros[n][i][np.isin(gnlct_values, boreal_range)] = peatland_df.loc[peatland_df['NGFBFC'] == ngfbfc, 'Boreal'].values[0]
            
            if np.any(np.isin(gnlct_values, temperate_range)):
                ef_zeros[n][i][np.isin(gnlct_values, temperate_range)] = peatland_df.loc[peatland_df['NGFBFC'] == ngfbfc, 'Temperate'].values[0]
            
            if np.any(np.isin(gnlct_values, tropical_range)):
                ef_zeros[n][i][np.isin(gnlct_values, tropical_range)] = peatland_df.loc[peatland_df['NGFBFC'] == ngfbfc, 'Tropical'].values[0]
        
    EF_netcdf = save_netcdf_ngfbfc(out_file_name=sheet, variabel=str(sheet), dims=dims_ngfbfc, input_data=ef_zeros, input_ngfbfc=gfrac_ngfbfc, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))

EF_CO2 = xr.open_dataset("/OUTPUT_DATA/CO2.NC", engine="netcdf4")
EF_CO2_eq = EF_CO2['CO2'] * tonne_to_tonne * C_to_CO2 

EF_CH4 = xr.open_dataset("/OUTPUT_DATA/CH4.NC", engine="netcdf4")
EF_CH4_eq = EF_CH4['CH4'] * Kg_to_tonne * CH4_to_CH4 

EF_N2O = xr.open_dataset("/OUTPUT_DATA/N2O.NC", engine="netcdf4")
EF_N2O_eq = EF_N2O['N2O'] * Kg_to_tonne * N_to_N2O 

EF_DOC = xr.open_dataset("/OUTPUT_DATA/DOC.NC", engine="netcdf4")
EF_DOC_eq = EF_DOC['DOC'] * tonne_to_tonne * C_to_CO2 

# S4.3B
emission_peat_CO2 = np.zeros((len_ngfbfc, analysis_period, lat, lon), dtype=std_dtype_float)
emission_peat_CH4 = np.zeros((len_ngfbfc, analysis_period, lat, lon), dtype=std_dtype_float)
emission_peat_N2O = np.zeros((len_ngfbfc, analysis_period, lat, lon), dtype=std_dtype_float)
emission_peat_DOC = np.zeros((len_ngfbfc, analysis_period, lat, lon), dtype=std_dtype_float)

for i, crops in enumerate(gfrac_ngfbfc):
    for n in range(analysis_period):
        emission_peat_CO2[i][n] = matrixMultiply(EF_CO2_eq[n], peatland_area_prop_crop[i][n])
        emission_peat_CH4[i][n] = matrixMultiply(EF_CH4_eq[n], peatland_area_prop_crop[i][n])
        emission_peat_N2O[i][n] = matrixMultiply(EF_N2O_eq[n], peatland_area_prop_crop[i][n])
        emission_peat_DOC[i][n] = matrixMultiply(EF_DOC_eq[n], peatland_area_prop_crop[i][n])
                                    
emission_peat_CO2_GtCO2eq = emission_peat_CO2 * CO2_to_CO2eq * ton_to_gigaton                       
emission_peat_CH4_GtCO2eq = emission_peat_CH4 * CH4_to_CO2eq * ton_to_gigaton 
emission_peat_N2O_GtCO2eq = emission_peat_N2O * N2O_to_CO2eq * ton_to_gigaton 
emission_peat_DOC_GtCO2eq = emission_peat_DOC * CO2_to_CO2eq * ton_to_gigaton 

emission_peat_CO2_GtCO2eq_netcdf = save_netcdf_variabel(out_file_name="emission_peat_CO2_GtCO2eq", variabel=gfrac_ngfbfc, input_data=emission_peat_CO2_GtCO2eq, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
emission_peat_CO2_GtCO2eq_netcdf = emission_peat_CO2_GtCO2eq_netcdf.assign_coords(country_name=country_coords["country"])
emission_peat_CO2_GtCO2eq_excel = netcdf_to_excel_peryear(out_file_name="emission_peat_CO2_GtCO2eq", input_data=emission_peat_CO2_GtCO2eq_netcdf, year=analysis_period, columns_name="NGFBFC", input_label=gfrac_ngfbfc, index_data=["country_name"], column_data=['time'], agg="sum")
emission_peat_CO2_GtCO2eq_excel.insert(3, "emissions", "peat_CO2")
emission_peat_CO2_GtCO2eq_excel.columns[:, columns_year_analysis] = (emission_peat_CO2_GtCO2eq_excel.columns[columns_year_analysis].values / interval)
emission_peat_CO2_GtCO2eq_excel.to_excel("/OUTPUT_DATA/emission_peat_CO2_GtCO2eq.xlsx", index=False)

emission_peat_CH4_GtCO2eq_netcdf = save_netcdf_variabel(out_file_name="emission_peat_CH4_GtCO2eq", variabel=gfrac_ngfbfc, input_data=emission_peat_CH4_GtCO2eq, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
emission_peat_CH4_GtCO2eq_netcdf = emission_peat_CH4_GtCO2eq_netcdf.assign_coords(country_name=country_coords["country"])
emission_peat_CH4_GtCO2eq_excel = netcdf_to_excel_peryear(out_file_name="emission_peat_CH4_GtCO2eq", input_data=emission_peat_CH4_GtCO2eq_netcdf, year=analysis_period, columns_name="NGFBFC", input_label=gfrac_ngfbfc, index_data=["country_name"], column_data=['time'], agg="sum")
emission_peat_CH4_GtCO2eq_excel.insert(3, "emissions", "peat_CH4")
emission_peat_CH4_GtCO2eq_excel.columns[:, columns_year_analysis] = (emission_peat_CH4_GtCO2eq_excel.columns[columns_year_analysis].values / interval)
emission_peat_CH4_GtCO2eq_excel.to_excel("/OUTPUT_DATA/emission_peat_CH4_GtCO2eq.xlsx", index=False)

emission_peat_N2O_GtCO2eq_netcdf = save_netcdf_variabel(out_file_name="emission_peat_N2O_GtCO2eq", variabel=gfrac_ngfbfc, input_data=emission_peat_N2O_GtCO2eq, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
emission_peat_N2O_GtCO2eq_netcdf = emission_peat_N2O_GtCO2eq_netcdf.assign_coords(country_name=country_coords["country"])
emission_peat_N2O_GtCO2eq_excel = netcdf_to_excel_peryear(out_file_name="emission_peat_N2O_GtCO2eq", input_data=emission_peat_N2O_GtCO2eq_netcdf, year=analysis_period, columns_name="NGFBFC", input_label=gfrac_ngfbfc, index_data=["country_name"], column_data=['time'], agg="sum")
emission_peat_N2O_GtCO2eq_excel.insert(3, "emissions", "peat_N2O")
emission_peat_N2O_GtCO2eq_excel.columns[:, columns_year_analysis] = (emission_peat_N2O_GtCO2eq_excel.columns[columns_year_analysis].values / interval)
emission_peat_N2O_GtCO2eq_excel.to_excel("/OUTPUT_DATA/emission_peat_N2O_GtCO2eq.xlsx", index=False)

emission_peat_DOC_GtCO2eq_netcdf = save_netcdf_variabel(out_file_name="emission_peat_DOC_GtCO2eq", variabel=gfrac_ngfbfc, input_data=emission_peat_DOC_GtCO2eq, coord=gfrac_combined_netcdf, time=pd.date_range(start=label_start_time_analysis, end=label_end_time_analysis, freq=frequensi))
emission_peat_DOC_GtCO2eq_netcdf = emission_peat_DOC_GtCO2eq_netcdf.assign_coords(country_name=country_coords["country"])
emission_peat_DOC_GtCO2eq_excel = netcdf_to_excel_peryear(out_file_name="emission_peat_DOC_GtCO2eq", input_data=emission_peat_DOC_GtCO2eq_netcdf, year=analysis_period, columns_name="NGFBFC", input_label=gfrac_ngfbfc, index_data=["country_name"], column_data=['time'], agg="sum")
emission_peat_DOC_GtCO2eq_excel.insert(3, "emissions", "peat_DOC")
emission_peat_DOC_GtCO2eq_excel.columns[:, columns_year_analysis] = (emission_peat_DOC_GtCO2eq_excel.columns[columns_year_analysis].values / interval)
emission_peat_DOC_GtCO2eq_excel.to_excel("/OUTPUT_DATA/emission_peat_DOC_GtCO2eq.xlsx", index=False)

emission_CO2_LUC_Category_AM1_excel = emission_CO2_LUC_Category_AM1_excel.drop(columns=['Country'])
emission_CO2_LUC_Category_AM1_excel = emission_CO2_LUC_Category_AM1_excel.rename(columns={"LUC_Category" : "emissions"})
emission_CO2_LUC_Category_AM1_excel.columns[:, columns_year_analysis] = emission_CO2_LUC_Category_AM1_excel.columns[columns_year_analysis].values / interval
emission_CO2_LUC_Category_AM1_excel_filtered = emission_CO2_LUC_Category_AM1_excel[emission_CO2_LUC_Category_AM1_excel['emissions'].isin(['Agricultural Transition Emission', 'LUC Crops Emission'])]
emission_CO2_LUC_Category_AM1_excel_filtered.to_excel("/OUTPUT_DATA//LUC2_Category.xlsx", index=False)

In [ ]:
## Step 5
# S5.A
df_physical = pd.read_excel("/INPUT_DATA/Physical to Harvested Area GFRAC.xlsx")

time = list(range(start_time_year, end_time_year, interval)) 

df1_copy = df_physical.copy()
df1_copy = df1_copy.rename(columns={"t" : "time"})
df1_copy = df1_copy[~df1_copy['NFBFC'].str.contains("biofuel")]
df1_copy.loc[:, 'RF/IR'] = np.where(np.arange(4192) % 32 < 16, "RF", "IR")
df1_copy = ngfbfc_processing(df1_copy)
df1_copy["NGFBFC"] = df1_copy['RF/IR'] + " " + df1_copy['NFBFC']
df1_copy = df1_copy.drop(columns=["NFBFC", "RF/IR"])
df1_melt = df1_copy.melt(id_vars=['NGFBFC', 'time'], var_name='IMAGE Region Name', value_name='value')

df2_copy = gfrac32_excel.copy()
df2_copy = ngfbfc_processing(df2_copy)
df2_merge = pd.merge(left=df2_copy, right=country_code, left_on="country", right_on="ISO Country")
df2_pivot = pd.pivot_table(data=df2_merge, values=time, index=['IMAGE Region Name', 'NGFBFC'], fill_value=0, aggfunc='sum')
df2_index = df2_pivot.reset_index()
df2_melt = df2_index.melt(id_vars=['IMAGE Region Name', 'NGFBFC'], var_name='time', value_name='value')

df_copy = df2_melt.copy()
for region, time, ngfbfc in zip(df_copy["IMAGE Region Name"].to_list(), df_copy["time"].to_list(), df_copy["NGFBFC"].to_list()):
    try:
        data1 = df1_melt[(df1_melt['IMAGE Region Name'] == region) & (df1_melt['time'] == time) & (df1_melt['NGFBFC'] == ngfbfc)]['value'].to_numpy()
        data2 = df2_melt[(df2_melt['IMAGE Region Name'] == region) & (df2_melt['time'] == time) & (df2_melt['NGFBFC'] == ngfbfc)]['value'].to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        mask = df_copy[(df_copy['IMAGE Region Name'] == region) & (df_copy['time'] == time) & (df_copy['NGFBFC'] == ngfbfc)].index
        df_copy.loc[mask, 'value'] =  data1[0] * data2[0]
    except KeyError as e:
        print(f"Error: {e} for region {region}")

df_copy["NGFBFC"] = df_copy["NGFBFC"].apply(lambda x: x.replace('IR ', '').replace('RF ', ''))
df_groupby = df_copy.groupby(['IMAGE Region Name', 'NGFBFC', 'time']).sum()
df_index = df_groupby.reset_index()

df_restructur = pd.pivot_table(data=df_index, columns=["time"], index=["IMAGE Region Name", "NGFBFC"], values=['value'])
restructur_index = df_restructur.stack(level=0, future_stack=True)
restructur_index = restructur_index.reset_index()
restructur_index = restructur_index.drop(columns=['level_2'])

restructur_index.to_csv("/OUTPUT_DATA/GFRAC_Area_Harvested_combined_kolom.csv", index=False)

# S5.B
fao_stat_ori = pd.read_csv("/INPUT_DATA/Production_Crops_Livestock_E_All_Data_analyze.csv")
crop_class_ori = pd.read_excel("/INPUT_DATA/Crop Classification_latest.xlsx", engine="openpyxl", skiprows=1)

crop_class_ori = crop_class_ori.drop('Unnamed: 0', axis=1)
crop_class_ori.rename(columns={'FAO Crops': 'Item'}, inplace=True)

fao_stat_ori = fao_stat_ori[~fao_stat_ori["Area"].isin(["Americas", "Asia", "Australia and New Zealand", "Africa", "Belgium-Luxembourg", "Central America", "Central Asia", "Caribbean", "Czechoslovakia", "Eastern Africa", "Eastern Asia",
 "Eastern Europe", "European Union (27)","Land Locked Developing Countries", "Least Developed Countries", "Europe", "Low Income Food Deficit Countries", "Melanesia", "Middle Africa", 
  "Net Food Importing Developing Countries", "Northern Africa", "Northern America", "Northern Europe", "Oceania", "Polynesia", "Small Island Developing States", "Serbia and Montenegro", "China",
  "South-eastern Asia", "Southern Africa", "Southern Asia", "Southern Europe", "Sudan (former)", "South America", "Western Africa", "Western Asia", "Western Europe", "World", "Yugoslav SFR"])]

fao_stat_production = fao_stat_ori[fao_stat_ori['Element'] == "Production"]
fao_stat_production = fao_stat_production.fillna(0)
fao_stat_production = fao_stat_production.merge(crop_class_ori, on='Item')

fao_stat_prod = fao_stat_production.drop_duplicates(subset=['Area', 'Item', 'Year', 'Element', 'IMAGE Classification'])
fao_stat_prod_columns = fao_stat_prod[['Area', 'Item', 'Year','IMAGE Classification', 'Value']]
mask = (fao_stat_prod_columns['Area'] == "Russian Federation") & (fao_stat_prod_columns['Year'] <= 1991)
ussr_mask = (fao_stat_prod_columns['Area'] == "USSR") & (fao_stat_prod_columns['Year'] <= 1991)

ussr_values = fao_stat_prod_columns.loc[ussr_mask, ['Item', 'Year', 'IMAGE Classification', 'Value']].values
fao_stat_prod_columns.loc[mask, ['Item', 'Year', 'IMAGE Classification', 'Value']] = ussr_values[:len(mask[mask])]
fao_stat_prod_columns_update = fao_stat_prod_columns[~fao_stat_prod_columns["Area"].isin(["USSR"])]
fao_stat_prod_columns_drop = fao_stat_prod_columns_update.drop_duplicates(subset=['Area', 'Item', 'Year', 'IMAGE Classification', 'Value'])

step1_copy = fao_stat_prod_columns_drop.copy()
step1_copy['Value'] = step1_copy['Value'].fillna(0)
step1_copy['Value'] = 0
step1_copy['Value'] = fao_stat_prod_columns_drop.groupby(['Area', "Year", 'IMAGE Classification'])['Value'].transform('sum')

step1_copy_drop = step1_copy.drop_duplicates(subset=['Area', 'Year', 'IMAGE Classification', 'Value'])
step1_copy_columns = step1_copy_drop[['Area', 'Year', 'IMAGE Classification', 'Value']]

eight_year_ranges = [(y, y+4) for y in range(1968, end_time_year, interval)]
step2_copy = step1_copy_columns.copy()
for year in eight_year_ranges:
    mask = (step1_copy_columns['Year'] >= year[0]) & (step1_copy_columns['Year'] <= year[1]) #5years
    step2_copy.loc[mask, 'Value'] = step1_copy_columns[mask].groupby(['Area', 'IMAGE Classification'])['Value'].transform('mean')

step2_copy_drop = step2_copy.drop_duplicates(subset=['Area', 'Year', 'IMAGE Classification', 'Value'])

eight_year_ranges = [(y, y+4) for y in range(1968, end_time_year, interval)]
five_yearIncrements = range(start_time_year, end_time_year, interval)
step3_copy = step2_copy_drop.copy()
for year, five_year in zip(eight_year_ranges, five_yearIncrements):
    year_idx = step2_copy_drop[(step2_copy_drop['Year'] >= year[0]) & (step2_copy_drop['Year'] <= year[1])]['Year'].index
    step3_copy.loc[year_idx, 'Year'] = five_year

step3_copy_drop = step3_copy.drop_duplicates(subset=['Area', 'Year', 'IMAGE Classification', 'Value'])

fao_stat_copy = step3_copy_drop.copy()
prubahan_nama = {
    "China, Hong Kong SAR": "Hong Kong",
    "China, Macao SAR": "Macao", 
    "China, Taiwan Province of": "Taiwan,  Province of China",
    "China, mainland": "China",
    "Democratic People's Republic of Korea": "Korea (Democratic People's Republic of)",
    "Democratic Republic of the Congo": "Congo, Democratic Republic of the",
    "Micronesia": "Micronesia (Federated States of)",
    "Micronesia (Federated States of) (Federated States of)": "Micronesia (Federated States of)",
    "Netherlands (Kingdom of the)": "Netherlands",
    "Republic of Korea": "Korea, Republic of",
    "Republic of Moldova": "Moldova, Republic of",
    "Palestine" : "Palestine, State of",
    "Türkiye": "Turkey",
    "United Republic of Tanzania": "Tanzania, United Republic of",
    "Ethiopia PDR" : "Eritrea"
}
keys_list = list(prubahan_nama.keys())
values_list = list(prubahan_nama.values())

for i in range(len(keys_list)):
    fao_stat_copy['Area'] = fao_stat_copy['Area'].astype(str).str.replace(keys_list[i], values_list[i])

fao_stat_copy.to_csv("/OUTPUT_DATA/REV_fao_stat_production_mov_average_v2.csv", index=False)

# S5b.1b
faostat_prod_1a = fao_stat_copy.rename(columns={"Value": "Fao Area Harvested", "IMAGE Classification": "NGFBFC", "Year": "time"})
faostat_prod_1a = pd.merge(left=faostat_prod_1a, right=country_code, left_on='Area', right_on='ISO Country')
faostat_prod_1a = faostat_prod_1a[['IMAGE Region Name', 'time', 'NGFBFC', 'Fao Area Harvested']]
faostat_prod_1a = faostat_prod_1a.groupby(['IMAGE Region Name', 'time', 'NGFBFC']).sum()
faostat_prod_1a = faostat_prod_1a.reset_index()
faostat_prod_1a.to_excel("/OUTPUT_DATA/REV_fao_stat_area_harvested_mov_average.xlsx", index=False)

# S5b.2b
faostat_prod_2a = fao_stat_copy.rename(columns={"Value": "FAO Production", "IMAGE Classification": "NGFBFC", "Year": "time"})
faostat_prod_2a = pd.merge(left=faostat_prod_2a, right=country_code, left_on='Area', right_on='ISO Country')
faostat_prod_2a = faostat_prod_2a[['IMAGE Region Name', 'time', 'NGFBFC', 'FAO Production']]
faostat_prod_2a = faostat_prod_2a.groupby(['IMAGE Region Name', 'time', 'NGFBFC']).sum()
faostat_prod_2a = faostat_prod_2a.reset_index()
faostat_prod_2a.to_excel("/OUTPUT_DATA/REV_fao_stat_production_mov_average_v2_REGION.xlsx", index=False)

# S5b.1c
faostat_area_harvested = pd.merge(left=faostat_prod_1a, right=country_code, left_on='Area', right_on='ISO Country')
faostat_area_harvested = faostat_area_harvested[['IMAGE Region Name', 'time', 'NGFBFC', 'Fao Area Harvested']]
faostat_area_harvested = faostat_area_harvested.groupby(['IMAGE Region Name', 'time', 'NGFBFC']).sum()
faostat_area_harvested = faostat_area_harvested.reset_index()
faostat_area_harvested.to_excel("/OUTPUT_DATA/REV_fao_stat_area_harvested_mov_average_REGION.xlsx", index=False)
faostat_area_harvested = faostat_area_harvested.rename(columns={"FAO Production": "Fao Area Harvested"})

gfrac32_concat_melt = gfrac32_excel.rename(columns={"value": "GFRAC"})
gfrac32_concat_melt = gfrac32_concat_melt.copy()
gfrac32_concat_melt.loc[:,'FAO'] = 0.0
gfrac32_concat_melt['FAO'] = gfrac32_concat_melt.groupby(['NGFBFC', 'time', 'IMAGE Region Name'])['GFRAC'].transform(lambda x: faostat_area_harvested[(faostat_area_harvested['NGFBFC'] == x.name[0]) & (faostat_area_harvested['time'] == x.name[1]) & (faostat_area_harvested['IMAGE Region Name'] == x.name[2])]['Fao Area Harvested'].sum())
gfrac32_concat_melt.loc[:, 'selisih'] = abs(gfrac32_concat_melt['GFRAC'] - gfrac32_concat_melt['FAO'])
gfrac32_concat_melt.sort_values(by='selisih', ascending=False)
gfrac32_concat_melt.to_excel("/OUTPUT_DATA/Komparasi_FAO_GFRAC.xlsx", index=False)

# S5b.2c
faostat_production = pd.merge(left=faostat_prod_2a, right=country_code, left_on='Area', right_on='ISO Country')
faostat_production = faostat_production[['IMAGE Region Name', 'time', 'NGFBFC', 'FAO Production']]
faostat_production = faostat_production.groupby(['IMAGE Region Name', 'time', 'NGFBFC']).sum()
faostat_production = faostat_production.reset_index()
faostat_production.to_excel("/OUTPUT_DATA/REV_fao_stat_area_harvested_mov_average_v2_REGION.xlsx", index=False)

grapc_concat_melted = gfrac32_excel.rename(columns={"GFRAC": "GRAPC"})
grapc_concat_melted = grapc_concat_melted.copy()
grapc_concat_melted.loc[:,'FAO'] = 0.0
grapc_concat_melted['FAO'] = grapc_concat_melted.groupby(['NGFBFC', 'time', 'IMAGE Region Name'])['GRAPC'].transform(lambda x: faostat_production[(faostat_production['NGFBFC'] == x.name[0]) & (faostat_production['time'] == x.name[1]) & (faostat_production['IMAGE Region Name'] == x.name[2])]['FAO Production'].sum())
grapc_concat_melted.loc[:, 'selisih'] = abs(grapc_concat_melted['GRAPC'] - grapc_concat_melted['FAO'])
grapc_concat_melted.sort_values(by='selisih', ascending=False)
grapc_concat_melted.to_excel("/OUTPUT_DATA/Komparasi_FAO_GRAPC.xlsx", index=False)

In [ ]:
GECH4RI = pd.read_excel("/OUTPUT_DATA/GECH4RI_crop_CH4.xlsx")
GECO2AB_CH4 = pd.read_excel("/OUTPUT_DATA/geCH4ab_crop_CH4.xlsx") 
GECO2AB_N2O = pd.read_excel("/OUTPUT_DATA/geN2Oab_crop_N2O.xlsx")
GEC2OLC = pd.read_excel("/OUTPUT_DATA/GEN2OLC_crop_N20.xlsx")
GEN2ORE = pd.read_excel("/OUTPUT_DATA/GEN2ORE_crop_v2.xlsx")
GFERTYN = pd.read_excel("/OUTPUT_DATA/GFERTILIZER_SYNTHETIC_crop_N20.xlsx")
GMANURE = pd.read_excel("/OUTPUT_DATA/GFERTILIZER_crop_N20.xlsx")

peat_CH4 = pd.read_excel("/OUTPUT_DATA/emission_peat_CH4_GtCO2eq.xlsx")
peat_CO2 = pd.read_excel("/OUTPUT_DATA/emission_peat_CO2_GtCO2eq.xlsx")
peat_DOC = pd.read_excel("/OUTPUT_DATA/emission_peat_DOC_GtCO2eq.xlsx")
peat_N2O = pd.read_excel("/OUTPUT_DATA/emission_peat_N2O_GtCO2eq.xlsx")

GECH4RI = ngfbfc_processing(GECH4RI)
GECO2AB_CH4 = ngfbfc_processing(GECO2AB_CH4)
GECO2AB_N2O = ngfbfc_processing(GECO2AB_N2O)
GEC2OLC = ngfbfc_processing(GEC2OLC)
GEN2ORE = ngfbfc_processing(GEN2ORE)
GFERTYN = ngfbfc_processing(GFERTYN)
GMANURE = ngfbfc_processing(GMANURE)
LUC_agri = ngfbfc_processing(luc_crops_prop_crop_AM1_excel)
agri_to_agri = ngfbfc_processing(agri_to_agri_prop_crop_AM1_excel)

peat_CH4 = ngfbfc_processing(peat_CH4)
peat_CO2 = ngfbfc_processing(peat_CO2)
peat_DOC = ngfbfc_processing(peat_DOC)
peat_N2O = ngfbfc_processing(peat_N2O)

all_sources = pd.concat([LUC_agri, agri_to_agri, GEC2OLC, GECH4RI, GECO2AB_CH4, GECO2AB_N2O, GEN2ORE, GFERTYN, GMANURE, peat_CH4, peat_CO2, peat_DOC, peat_N2O], ignore_index=True)
all_sources.columns[:, range(start_time_year+5, end_time_year, interval)] = all_sources.columns[range(start_time_year+5, end_time_year, interval)].values * 5

all_sources_copy = all_sources.copy()
all_sources_copy.rename(columns={'emissions': 'Process'}, inplace=True)
all_sources_copy['Process'] = all_sources_copy['Process'].str.replace("peat_DOC", 'peat_CO2')

In [ ]:
tahun = []
for n in range(start_time_year+5, end_time_year, interval):
    n_awal = n - interval
    tahun.append(f'{str(n_awal)[2:]}-{str(n)[2:]}')

all_sources_process = all_sources_copy.groupby(['Process']).sum()
all_sources_process_reset = all_sources_process.reset_index()

all_sources_process_drop = all_sources_process_reset.drop(columns=['IMAGE Region Name','NGFBFC'])
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('LUC_Agri', 'Natural Vegetation to Agriculture Emission')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('agri2agri', 'Agricultural Transition Emission')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('GEN2OLC', 'N2O Land Clearing')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('GECH4RI', 'CH4 Wetland Rice')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('agri_burn_CH4', 'CH4 Agriculture Waste Burning')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('agri_burn_N2O', 'N2O Agriculture Waste Burning')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('GEN2ORE', 'N2O Agriculture Residues')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('GFERTSYN', 'N2O Synthetic Fertilizer')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('GMANURE', 'N2O Manure Fertilizer')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('peat_CH4', 'CH4 Drained Peatland')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('peat_CO2', 'CO2 Drained Peatland')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('peat_CH4', 'CH4 Drained Peatland')
all_sources_process_drop['Process'] = all_sources_process_drop['Process'].replace('peat_N2O', 'N2O Drained Peatland')

new_columns_order_process = [6,5,1,9,10,11,3,4,7,8,2,0]
all_sources_process_reordered = all_sources_process_drop.reindex(new_columns_order_process)
all_sources_process_reordered = all_sources_process_reordered.reset_index()
all_sources_process_reordered = all_sources_process_reordered.drop(columns = "index")
all_sources_process_reordered_index = all_sources_process_reordered.set_index('Process')
value_all_sources_process = all_sources_process_reordered_index.T.values

panel_A = pd.DataFrame(value_all_sources_process, index=tahun, columns=columns1)
ax1 = panel_A.plot(kind='bar', stacked=True, color=colors1, figsize=(14, 8), legend=False)

plt.xlabel('Year', fontweight='bold', fontsize=15)
plt.ylabel(r"$\bf{Carbon\ Emissions\ (Gt~CO_2 eq.)/\ year}$", fontsize=15)
plt.xticks([r for r in range(len(tahun))], tahun, rotation=0)
plt.axhline(0, color='black', linewidth=0.8)

yticks = np.arange(0, 6, 1)
plt.yticks(yticks)

handles = []
handles, labels = ax1.get_legend_handles_labels()
labels.append('Average annual emission (Gt CO2/ year)')

order1 = [11,10,9,8,7,6,5,4,3,2,1,0]
ax1.legend([handles[idx] for idx in order1], [labels[idx] for idx in order1], title='Emissions Category', loc='upper left', bbox_to_anchor=(1.01, 1.0), ncol=1)
plt.show()
plt.savefig('/VIZ/Grafik_panelA.png')

all_source_crops = all_sources_copy.groupby(['NGFBFC']).sum()
all_source_crops_reset = all_source_crops.reset_index()
all_source_crops_drop = all_source_crops_reset.drop(columns=['IMAGE Region Name','Process'])
all_source_crops_drop['NGFBFC'] = all_source_crops_drop['NGFBFC'].replace('grass', 'pasture')
all_source_crops_drop = ngfbfc_processing(all_source_crops_drop)

all_source_crops_drop.set_index('NGFBFC', inplace=True)
all_source_crops_reorder = all_source_crops_drop.reindex(list(colors.keys()))
value_all_sources_crops = all_source_crops_reorder.T.values

panel_B = pd.DataFrame(value_all_sources_crops, index=tahun, columns=list(colors.keys()))
ax2 = panel_B.plot(kind='bar', stacked=True, color=list(colors.values()), figsize=(14, 8), legend=False)
plt.xlabel('Year', fontweight='bold', fontsize=15)
plt.ylabel(r"$\bf{Carbon\ Emissions\ (Gt~CO_2)}$", fontsize=15)
plt.xticks([r for r in range(len(tahun))], tahun, rotation=360)
plt.axhline(0, color='black', linewidth=0.8)

handles, labels = ax2.get_legend_handles_labels()
labels.append('Average annual emission (Gt CO2/ year)')

order2 = [16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0]
ax2.legend([handles[idx] for idx in order2], [labels[idx] for idx in order2], title='Crop categories', loc='upper right', bbox_to_anchor=(1.23, 1.0), ncol=1)
plt.show()
plt.savefig('/VIZ/Grafik_panelB.png')

all_sources_region = all_sources_copy.groupby(['IMAGE Region Name']).sum()
all_sources_region_reset = all_sources_region.reset_index()
all_sources_region_drop = all_sources_region_reset.drop(columns=['Process','NGFBFC'])
all_sources_region_drop = region_processing(all_sources_region_drop)
all_sources_region_drop = all_sources_region_drop.set_index("IMAGE Region Name")
all_source_region_reorder = all_sources_region_drop.reindex(list(palettes.keys()))
value_all_sources_region = all_source_region_reorder.T.values

panel_C = pd.DataFrame(value_all_sources_region, index=tahun, columns=list(palettes.keys()))
ax3 = panel_C.plot(kind='bar', stacked=True, color=list(palettes.values()), figsize=(14, 8), legend=False)

plt.xlabel('Year', fontweight='bold', fontsize=15)
plt.ylabel(r"$\bf{GHG\ Emissions\ (Gt~CO_2-eq.)/\ year}$", fontsize=15)
plt.xticks([r for r in range(len(tahun))], tahun, rotation=360)
plt.axhline(0, color='black', linewidth=0.8)

handles, labels = ax3.get_legend_handles_labels()
order3 = [25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0]
ax3.legend([handles[idx] for idx in order3], [labels[idx] for idx in order3], title='IMAGE Region', loc='upper right', bbox_to_anchor=(1.22, 1.0), ncol=1)
plt.show()
plt.savefig('/VIZ/Grafik_panelC.png')

fig, axs = plt.subplots(3, 1, figsize=(13, 18))
axs[0] = panel_A.plot(kind='bar', stacked=True, color=colors1, ax=axs[0], legend=False)
axs[0].set_ylabel(r"$\bf{GHG\ Emissions\ (Gt~CO_2-eq.)/\ year}$", fontsize=15)
axs[0].set_xticks([r for r in range(len(tahun))])
axs[0].set_xticklabels(tahun, rotation=0)
axs[0].axhline(0, color='black', linewidth=0.8)

axs[1] = panel_B.plot(kind='bar', stacked=True, color=list(colors.values()), ax=axs[1], legend=False)
axs[1].set_ylabel(r"$\bf{GHG\ Emissions\ (Gt~CO_2-eq.)/\ year}$", fontsize=15)
axs[1].set_xticks([r for r in range(len(tahun))])
axs[1].set_xticklabels(tahun, rotation=360)
axs[1].axhline(0, color='black', linewidth=0.8)

axs[2] = panel_C.plot(kind='bar', stacked=True, color=list(palettes.values()), ax=axs[2], legend=False)
axs[2].set_ylabel(r"$\bf{GHG\ Emissions\ (Gt~CO_2-eq.)/\ year}$", fontsize=15)
axs[2].set_xticks([r for r in range(len(tahun))])
axs[2].set_xticklabels(tahun, rotation=360)
axs[2].axhline(0, color='black', linewidth=0.8)
axs[2].set_xlabel('Year', fontweight='bold', fontsize=15)

bbox_transform = fig.transFigure
plt.rc('text', usetex=False)

handles1, labels1 = axs[0].get_legend_handles_labels()
axs[0].legend([handles1[idx] for idx in order1], [labels1[idx] for idx in order1] ,edgecolor='white',loc='upper left', bbox_to_anchor=(1.005, 0.95))

handles2, labels2 = axs[1].get_legend_handles_labels()
axs[1].legend([handles2[idx] for idx in order2], [labels2[idx] for idx in order2], edgecolor='white', loc='upper left', bbox_to_anchor=(1.005, 1), ncol=1)

handles3, labels3 = axs[2].get_legend_handles_labels()
axs[2].legend([handles3[idx] for idx in order3], [labels3[idx] for idx in order3],edgecolor='white', loc='upper left', bbox_to_anchor=(1.005, 1), ncol=1)

plt.subplots_adjust(hspace=0.001)
plt.tight_layout()
plt.show()
plt.savefig('/VIZ/Grafik_panelABC.png')

In [ ]:
all_sources_copy = all_sources[all_sources['NGFBFC'] != 'pasture']

df_LUC = all_sources_copy[(all_sources_copy['emissions'] == 'LUC_Agri') | (all_sources_copy['emissions'] == 'agri2agri') | (all_sources_copy['emissions'] == 'GEN2OLC')]
df_LUC = df_LUC.groupby(['IMAGE Region Name', 'NGFBFC']).sum()
df_LUC = df_LUC.reset_index()
df_LUC = df_LUC.drop(columns=['emissions'])
df_LUC.columns[:, range(start_time_year+5, end_time_year, interval)] = (df_LUC.columns[range(start_time_year+5, end_time_year, interval)].values * 5)

df_LUC_melt = df_LUC.melt(id_vars=['IMAGE Region Name', 'NGFBFC'], var_name='time', value_name='value')
df_LUC_melt.loc[:, 'emissions'] = "LUC"

df_AGRI = all_sources_copy[~(all_sources_copy['emissions'] == 'LUC_Agri') | ~(all_sources_copy['emissions'] == 'agri2agri') | ~(all_sources_copy['emissions'] == 'GEN2OLC')]
df_AGRI = df_AGRI.groupby(['IMAGE Region Name', 'NGFBFC']).sum()
df_AGRI = df_AGRI.reset_index()
df_AGRI = df_AGRI.drop(columns=['emissions'])
df_AGRI.columns[:, range(start_time_year+5, end_time_year, interval)] = (df_AGRI.columns[range(start_time_year+5, end_time_year, interval)].values * 5)

df_AGRI_melt = df_AGRI.melt(id_vars=['IMAGE Region Name', 'NGFBFC'], var_name='time', value_name='value')
df_AGRI_melt.loc[:, 'emissions'] = "AGRI"

data_2kategori = pd.concat([df_LUC_melt, df_AGRI_melt], ignore_index=True)
data_2kategori_drop = data_2kategori.drop(columns=['emissions'])
data_2kategori = data_2kategori_drop.groupby(columns).sum()
data_2kategori = data_2kategori.reset_index()

region_code_map = country_code.set_index('IMAGE Region Name')['IMAGE Region Code'].to_dict()
data_2kategori['IMAGE Region Name'] = data_2kategori['IMAGE Region Name'].map(region_code_map)

for time in range(start_time_year+5, end_time_year, interval):
    data_2kategori = data_2kategori[data_2kategori['time'] == time]

    df_top3 = get_top_n(data_2kategori, 'NGFBFC', 'value', 3)
    total_top3 = df_top3.groupby('NGFBFC')['value'].sum()
    total_top3 = total_top3.reset_index()

    total_per_NGFBFC = data_2kategori.groupby('NGFBFC')['value'].sum()
    total_per_NGFBFC = total_per_NGFBFC.reset_index()
    df_others_sum = total_per_NGFBFC.merge(total_top3, on='NGFBFC', how='left', suffixes=('', '_top3'))
    df_others_sum['value'] = df_others_sum['value'] - df_others_sum['value_top3'].fillna(0)
    df_others_sum = df_others_sum[df_others_sum['value'] > 0] 
    df_others_sum['IMAGE Region Name'] = "others"

    df_combined = pd.concat([df_top3, df_others_sum[['NGFBFC', 'value', 'IMAGE Region Name']]], ignore_index=True)
    df_combined = df_combined[df_combined['NGFBFC'] != 'pasture']

    df_combined['color_order'] = df_combined['NGFBFC'].map(lambda x: list(colors.keys()).index(x) if x in colors else len(colors))
    df_combined = df_combined.sort_values('color_order').drop('color_order', axis=1)

    ring1_data = df_combined.groupby('NGFBFC')['value'].sum()
    ring1_labels = [i for i in colors.keys() if i != 'pasture']
    ring1_data = ring1_data.reindex(ring1_labels)
    ring1_colors = [colors[label] for label in ring1_labels]

    ring1_percent = ring1_data / ring1_data.sum() * 100

    ring2_data = df_combined.groupby(['NGFBFC', 'IMAGE Region Name'])['value'].sum()

    NGFBFCs_list = list(colors.keys())
    final_data = []

    for NGFBFC_name in NGFBFCs_list:
        if NGFBFC_name in ring2_data.index.get_level_values(0):
            NGFBFC_data = ring2_data.xs(NGFBFC_name)
            angle = get_angular_position(NGFBFC_name, NGFBFCs_list)
            
            others_value = NGFBFC_data['others'] if 'others' in NGFBFC_data.index else None
            non_others = NGFBFC_data[NGFBFC_data.index != 'others'].sort_values(ascending=False)
            
            if 300 <= angle or angle <= 70:
                non_others = NGFBFC_data[NGFBFC_data.index != 'others'].sort_values(ascending=True)
                if others_value is not None:
                    regions = ['others'] + non_others.index.tolist()
                    values = [others_value] + non_others.values.tolist()
                else:
                    regions = non_others.index.tolist()
                    values = non_others.values.tolist()
            else:
                if others_value is not None:
                    regions = non_others.index.tolist() + ['others']
                    values = non_others.values.tolist() + [others_value]
                else:
                    regions = non_others.index.tolist()
                    values = non_others.values.tolist()
            
            for region, value in zip(regions, values):
                final_data.append(((NGFBFC_name, region), value))

    ring2_data = pd.Series(dict(final_data))
    ring2_labels = ring2_data.index.get_level_values(1)

    ring2_colors = []
    for NGFBFC_name in colors.keys():
        if NGFBFC_name in ring2_data.index.get_level_values(0):
            NGFBFC_data = ring2_data.xs(NGFBFC_name)
            ring2_colors.extend([colors[NGFBFC_name]] * len(NGFBFC_data))

    fig = plt.figure(figsize=(15, 10))
    gs = gridspec.GridSpec(2, 2, figure=fig, hspace=0.2, wspace=0.2, width_ratios=[7,3], height_ratios=[7,3])
    ax = fig.add_subplot(gs[:, 0], aspect="equal")

    plt.title(f"{time}")

    inner_pie, _ = ax.pie(ring1_data, radius=1.2, colors=ring1_colors, wedgeprops=dict(width=1, edgecolor='#808080'), startangle=90)
    outer_pie, _ = ax.pie(ring2_data, radius=1.5, colors=ring2_colors, wedgeprops=dict(width=0.7, edgecolor='#808080'), startangle=90)

    for i, (label, percent) in enumerate(zip(ring1_labels, ring1_percent)):
        ang = (inner_pie[i].theta2 + inner_pie[i].theta1) / 2
        x = 0.5 * np.cos(np.deg2rad(ang))
        y = 0.5 * np.sin(np.deg2rad(ang))
        rotation = ang + 180 if 90 <= ang <= 270 else ang
        ax.text(x, y, f'{percent:.1f}%', ha='center', va='center', rotation=rotation, 
                color='black', fontsize=14, fontweight='bold',
                path_effects=[path_effects.Stroke(linewidth=3, foreground='white'), 
                            path_effects.Normal()])

    for i, label in enumerate(ring2_labels):
        ang = (outer_pie[i].theta2 + outer_pie[i].theta1) / 2
        x = 1.2 * np.cos(np.deg2rad(ang))
        y = 1.2 * np.sin(np.deg2rad(ang))
        rotation = ang + 180 if 90 <= ang <= 270 else ang
        ax.text(x, y, f'{label}', ha='center', va='center', rotation=rotation,
                color='black', fontsize=12, fontweight='bold',
                path_effects=[path_effects.Stroke(linewidth=4, foreground='white'),
                            path_effects.Normal()])

    legend_handles = [plt.Rectangle((0, 0), 1, 1, edgecolor='#808080', facecolor=color) for color in dict(reversed(colors.items())).values()]
    
    legend_ax = fig.add_subplot(gs[0, 1])
    legend_ax.axis('off') 
    legend_ax.legend(handles=legend_handles,
            labels=[label for label in dict(reversed(colors.items())).keys()],
            title="Crop Categories", title_fontsize=14,
            bbox_to_anchor=(0,0.1), loc="lower left", fontsize=12)
    
    plt.show()
    plt.savefig(f'/VIZ/PieChart_multiyear_{time}.png')

df_AGRI_melt = df_AGRI.melt(id_vars=['IMAGE Region Name', 'NGFBFC'], var_name='time', value_name='value')
df_AGRI_melt = df_AGRI_melt.drop(columns=['time'])
df_AGRI_groupby = df_AGRI_melt.groupby(['IMAGE Region Name', 'NGFBFC']).sum()
df_AGRI_index = df_AGRI_groupby.reset_index()
df_AGRI_index.loc[:, 'emissions'] = "AGRI"

df_LUC_melt = df_LUC.melt(id_vars=['IMAGE Region Name', 'NGFBFC'], var_name='time', value_name='value')
df_LUC_melt = df_LUC_melt.drop(columns=['time'])
df_LUC_groupby = df_LUC_melt.groupby(['IMAGE Region Name', 'NGFBFC']).sum()
df_LUC_index = df_LUC_groupby.reset_index()
df_LUC_index.loc[:, 'emissions'] = "LUC"

data_2kategori = pd.concat([df_AGRI_index, df_LUC_index], ignore_index=True)
data_2kategori_drop = data_2kategori.drop(columns=['emissions'])
data_2kategori_sum = data_2kategori_drop.groupby(['IMAGE Region Name', 'NGFBFC']).sum()
data_2kategori_sum = data_2kategori_sum.reset_index()

region_code_map = country_code.set_index('IMAGE Region Name')['IMAGE Region Code'].to_dict()
data_2kategori_sum['IMAGE Region Name'] = data_2kategori_sum['IMAGE Region Name'].map(region_code_map)
total_per_NGFBFC = data_2kategori_sum.groupby('NGFBFC')['value'].sum()
total_per_NGFBFC = total_per_NGFBFC.reset_index()

df_top3 = get_top_n(data_2kategori_sum, 'NGFBFC', 'value', 3)
total_top3 = df_top3.groupby('NGFBFC')['value'].sum()
total_top3 = total_top3.reset_index()

df_others_sum = total_per_NGFBFC.merge(total_top3, on='NGFBFC', how='left', suffixes=('', '_top3'))
df_others_sum['value'] = df_others_sum['value'] - df_others_sum['value_top3'].fillna(0)
df_others_sum = df_others_sum[df_others_sum['value'] > 0]  
df_others_sum['IMAGE Region Name'] = "others"

df_combined = pd.concat([df_top3, df_others_sum[['NGFBFC', 'value', 'IMAGE Region Name']]], ignore_index=True)
df_combined = df_combined[df_combined['NGFBFC'] != 'grass']

df_combined['color_order'] = df_combined['NGFBFC'].map(lambda x: list(colors.keys()).index(x))
df_combined = df_combined.sort_values('color_order').drop('color_order', axis=1)

ring1_data = df_combined.groupby('NGFBFC')['value'].sum()
ring1_labels = [i for i in colors.keys() if i != 'pasture']
ring1_data = ring1_data.reindex(ring1_labels)
ring1_colors = [colors[label] for label in ring1_labels]

ring1_percent = ring1_data / ring1_data.sum() * 100
ring2_data = df_combined.groupby(['NGFBFC', 'IMAGE Region Name'])['value'].sum()

NGFBFCs_list = list(colors.keys())
final_data = []
for NGFBFC_name in NGFBFCs_list:
    if NGFBFC_name in ring2_data.index.get_level_values(0):
        NGFBFC_data = ring2_data.xs(NGFBFC_name)
        angle = get_angular_position(NGFBFC_name, NGFBFCs_list)
        
        others_value = NGFBFC_data['others'] if 'others' in NGFBFC_data.index else None
        non_others = NGFBFC_data[NGFBFC_data.index != 'others'].sort_values(ascending=False)
        
        if 300 <= angle or angle <= 70:
            non_others = NGFBFC_data[NGFBFC_data.index != 'others'].sort_values(ascending=True)
            if others_value is not None:
                regions = ['others'] + non_others.index.tolist()
                values = [others_value] + non_others.values.tolist()
            else:
                regions = non_others.index.tolist()
                values = non_others.values.tolist()
        else:
            if others_value is not None:
                regions = non_others.index.tolist() + ['others']
                values = non_others.values.tolist() + [others_value]
            else:
                regions = non_others.index.tolist()
                values = non_others.values.tolist()
        
        for region, value in zip(regions, values):
            final_data.append(((NGFBFC_name, region), value))

ring2_data = pd.Series(dict(final_data))
ring2_labels = ring2_data.index.get_level_values(1)

ring2_colors = []
for NGFBFC_name in colors.keys():
    if NGFBFC_name != 'pasture':
        if NGFBFC_name in ring2_data.index.get_level_values(0):
            NGFBFC_data = ring2_data.xs(NGFBFC_name)
            ring2_colors.extend([colors[NGFBFC_name]] * len(NGFBFC_data))

fig = plt.figure(figsize=(15, 10))
gs = gridspec.GridSpec(2, 2, figure=fig, hspace=0.2, wspace=0.2, width_ratios=[7,3], height_ratios=[7,3])
ax = fig.add_subplot(gs[:, 0], aspect="equal")

inner_pie, _ = ax.pie(ring1_data, radius=1.2, colors=ring1_colors, wedgeprops=dict(width=1, edgecolor='#808080'), startangle=90)
outer_pie, _ = ax.pie(ring2_data, radius=1.5, colors=ring2_colors, wedgeprops=dict(width=0.7, edgecolor='#808080'), startangle=90)

for i, (label, percent) in enumerate(zip(ring1_labels, ring1_percent)):
    ang = (inner_pie[i].theta2 + inner_pie[i].theta1) / 2
    x = 0.5 * np.cos(np.deg2rad(ang))
    y = 0.5 * np.sin(np.deg2rad(ang))
    rotation = ang + 180 if 90 <= ang <= 270 else ang
    ax.text(x, y, f'{percent:.1f}%', ha='center', va='center', rotation=rotation, 
            color='black', fontsize=14, fontweight='bold',
            path_effects=[path_effects.Stroke(linewidth=3, foreground='white'), 
                        path_effects.Normal()])

for i, label in enumerate(ring2_labels):
    ang = (outer_pie[i].theta2 + outer_pie[i].theta1) / 2
    x = 1.2 * np.cos(np.deg2rad(ang))
    y = 1.2 * np.sin(np.deg2rad(ang))
    rotation = ang + 180 if 90 <= ang <= 270 else ang
    ax.text(x, y, f'{label}', ha='center', va='center', rotation=rotation,
            color='black', fontsize=12, fontweight='bold',
            path_effects=[path_effects.Stroke(linewidth=4, foreground='white'),
                        path_effects.Normal()])
        
legend_handles = [plt.Rectangle((0, 0), 1, 1, edgecolor='#808080', facecolor=color) for color in dict(reversed(colors.items())).values()]
plt.legend(handles=legend_handles,
           labels=[label for label in dict(reversed(colors.items())).keys()],
           title="Crop Categories",title_fontsize=14,
           bbox_to_anchor=(1.1, 1), loc="upper left", fontsize=12)
plt.show()
plt.savefig(f'/VIZ/PieChart_akumulasi_{start_time_year+5}-{end_time_year}.png')

In [ ]:
all_sources_group = all_sources_copy.groupby(['IMAGE Region Name', 'NGFBFC']).sum()
all_sources_group = all_sources_group.reset_index()
all_sources_group = region_processing(all_sources_group)
all_sources_group = ngfbfc_processing(all_sources_group)
all_sources_group = all_sources_group[all_sources_group['NGFBFC'] != 'pasture']

all_sources_group_copy = all_sources_group.copy()
all_sources_group_copy.loc[:, range(start_time_year+5, end_time_year, interval)] = all_sources_group_copy[range(start_time_year+5, end_time_year, interval)].values * 1e3
all_sources_group_copy['IMAGE Realtion'] =  all_sources_group_copy['NGFBFC'] + " - "  + all_sources_group_copy['IMAGE Region Name']
all_sources_group_copy = all_sources_group_copy[list(range(start_time_year+5, end_time_year, interval)) + ['IMAGE Realtion']]

df_crops = all_sources_group_copy.melt(id_vars=['IMAGE Realtion'], var_name='Time', value_name='value')
df_crops = df_crops[df_crops['IMAGE Realtion'] != 'wheat - Rest of South Asia']

results_crops = {}
df_crops_sort = df_crops.sort_values(by=['value'], ascending=[True])
for time in df_crops_sort['Time'].unique():
    filtered_data = df_crops_sort[df_crops_sort['Time'] == time]
    sorted_data = filtered_data.sort_values(by='value', ascending=False)
    top_5_data = sorted_data.head(3)
    if time not in results_crops:
        results_crops[time] = {}
    results_crops[time] = top_5_data

time_map = {}
for n in range(start_time_year+5, end_time_year, interval):
    n_awal = n - interval
    time_map[n] = f'{str(n_awal)[2:]}-{str(n)[2:]}'

df_crops_concat = pd.concat([pd.concat(results_crops.values())])
df_crops_concat_copy = df_crops_concat.copy()
df_crops_concat_copy['Time'] = df_crops_concat_copy['Time'].map(time_map)
df_crops_concat_copy_sorted = df_crops_concat_copy.sort_values(by=['Time', 'value'], ascending=[False, False])

plt.figure(figsize=(25, 6), dpi=300)
loc = plticker.MultipleLocator(base=1)
ax2 = plt.subplot(131)
ax2.set_ylabel(r"$\bf{GHG\ Emissions\ (Mt~CO_2-eq)/\ year}$", fontsize=10)
ax2.set_xlabel('Years',fontweight='bold', fontsize=10)

sns.scatterplot(x="Time", y="value", color='#DDDDDD', sizes=(100, 300), alpha=.3, data=df_crops)
sns.scatterplot(x="Time", y="value", hue='IMAGE Realtion',alpha=.8, palette=palette, style='IMAGE Realtion', markers=markers, s=60, 
                data=df_crops_concat_copy_sorted.sort_values(by=['IMAGE Realtion'], ascending=[True]), hue_order=legend_order)

ax2.legend(loc='lower left', bbox_to_anchor= (1.01, 0.45), ncol=1, prop={'size': 8})
ax2.xaxis.set_major_locator(loc)
ax2.set_ylim(ymin=0)

plt.show()
plt.savefig(f'/VIZ/scatterplot_{start_time_year+5}-{end_time_year}.png')

In [ ]:
columns = ['IMAGE Region Name', 'NGFBFC'] + [x for x in range(start_time_year+5, end_time_year, interval)]

df_all_sources_copy = all_sources.copy()
df_all_sources_copy = ngfbfc_processing(df_all_sources_copy)
df_all_sources_copy = df_all_sources_copy[~df_all_sources_copy['NGFBFC'].str.contains("pasture")]

df_all_sources_copy = df_all_sources_copy[columns]
df_all_sources_copy = df_all_sources_copy.groupby(['IMAGE Region Name', 'NGFBFC']).sum()
df_all_sources_copy = df_all_sources_copy.reset_index()
df_all_sources_melt = df_all_sources_copy.melt(id_vars=['IMAGE Region Name', 'NGFBFC'], var_name='time', value_name='value')
df_all_sources_melt['value'] = df_all_sources_melt['value'] * 1e9

df_GFRACcombined_copy = gfrac_area_7020_excel.copy()
df_GFRACcombined_copy = ngfbfc_processing(df_GFRACcombined_copy)
df_GFRACcombined_copy = df_GFRACcombined_copy[~df_GFRACcombined_copy['NGFBFC'].str.contains("grass")]

df_GFRACcombined_copy = pd.merge(left=country_code, right=df_GFRACcombined_copy, how="left", left_on="ISO Country", right_on="country")
df_GFRACcombined_copy = df_GFRACcombined_copy[columns]
df_GFRACcombined_copy = df_GFRACcombined_copy[~(df_GFRACcombined_copy['IMAGE Region Name'] == "N|A")]
df_GFRACcombined_copy = df_GFRACcombined_copy.groupby(['IMAGE Region Name', 'NGFBFC']).sum()
df_GFRACcombined_copy = df_GFRACcombined_copy.reset_index()
df_GFRACcombined_melt = pd.melt(df_GFRACcombined_copy, id_vars=['IMAGE Region Name', 'NGFBFC'], var_name='time', value_name='value')

CF_emisi_total_GFRAC_area = df_all_sources_melt.copy()
CF_emisi_total_GFRAC_area = CF_emisi_total_GFRAC_area.drop(columns=['value'])

IMAGERegionName	= CF_emisi_total_GFRAC_area["IMAGE Region Name"].to_list()
time = CF_emisi_total_GFRAC_area["time"].to_list()
ngfbfc = CF_emisi_total_GFRAC_area["NGFBFC"].to_list()
for region, time, ngfbfc in zip(IMAGERegionName, time, ngfbfc):
    try:
        data1 = df_all_sources_melt[(df_all_sources_melt['IMAGE Region Name'] == region) & (df_all_sources_melt['time'] == time) & (df_all_sources_melt['NGFBFC'] == ngfbfc)]['value'].to_numpy()
        data2 = df_GFRACcombined_melt[(df_GFRACcombined_melt['IMAGE Region Name'] == region) & (df_GFRACcombined_melt['time'] == time) & (df_GFRACcombined_melt['NGFBFC'] == ngfbfc)]['value'].to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        mask = df_all_sources_melt[(df_all_sources_melt['IMAGE Region Name'] == region) & (df_all_sources_melt['time'] == time) & (df_all_sources_melt['NGFBFC'] == ngfbfc)].index
        CF_emisi_total_GFRAC_area.loc[mask, 'CF'] = data1[0] / data2[0]
    except KeyError as e:
        print(f"Error: {e} for region {region}")

CF_emisi_total_GFRAC_area['CF'] = CF_emisi_total_GFRAC_area['CF'].replace([np.inf, -np.inf], np.nan).fillna(0)

df_GFRAC_Area_copy = restructur_index.copy()
df_GFRAC_Area_copy = df_GFRAC_Area_copy[df_GFRAC_Area_copy['time'] >= start_time_year+5]
df_GFRAC_Area_copy = ngfbfc_processing(df_GFRAC_Area_copy)
df_GFRAC_Area_selected = df_GFRAC_Area_copy[~df_GFRAC_Area_copy['NGFBFC'].str.contains("grass")]

CF_emisi_total_AH = df_all_sources_melt.copy()
CF_emisi_total_AH = CF_emisi_total_AH.drop(columns=['value'])

IMAGERegionName = CF_emisi_total_AH["IMAGE Region Name"].to_list()
time = CF_emisi_total_AH["time"].to_list()
ngfbfc = CF_emisi_total_AH["NGFBFC"].to_list()
for region, time, ngfbfc in zip(IMAGERegionName, time, ngfbfc):
    try:
        data1 = df_all_sources_melt[(df_all_sources_melt['IMAGE Region Name'] == region) & (df_all_sources_melt['time'] == time) & (df_all_sources_melt['NGFBFC'] == ngfbfc)]['value'].to_numpy()
        data2 = df_GFRAC_Area_selected[(df_GFRAC_Area_selected['IMAGE Region Name'] == region) & (df_GFRAC_Area_selected['time'] == time) & (df_GFRAC_Area_selected['NGFBFC'] == ngfbfc)]['value'].to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        mask = df_all_sources_melt[(df_all_sources_melt['IMAGE Region Name'] == region) & (df_all_sources_melt['time'] == time) & (df_all_sources_melt['NGFBFC'] == ngfbfc)].index
        CF_emisi_total_AH.loc[mask, 'CF'] = data1[0] / data2[0]
    except KeyError as e:
        print(f"Error: {e} for region {region}")

CF_emisi_total_AH['CF'] = CF_emisi_total_AH['CF'].replace([np.inf, -np.inf], np.nan).fillna(0)

df1 = CF_emisi_total_AH.copy()
df2 = df_GFRAC_Area_selected.copy()

df1.loc[df1['CF'] > 50, 'CF'] = 0
df1 = ngfbfc_processing(df1)
df2 = ngfbfc_processing(df2)

data1 = np.zeros((len(list(range(start_time_year+5, end_time_year, interval))), len(df1['NGFBFC'].unique()), len(df2['IMAGE Region Name'].unique())), dtype='float32')
average_AreaHarvested_RegionCrops = np.zeros((len(list(range(start_time_year+5, end_time_year, interval))), len(df1['NGFBFC'].unique()), len(df2['IMAGE Region Name'].unique())), dtype='float32')
result_average = np.zeros((len(df1['NGFBFC'].unique()), len(list(range(start_time_year+5, end_time_year, interval)))), dtype='float32')
for n, time in enumerate(range(start_time_year+5, end_time_year, interval)):
    for i, ngfbfc in enumerate(df1['NGFBFC'].unique()):
        for j, image_region in enumerate(df2['IMAGE Region Name'].unique()):
            ngfbfc_areaharves = df2[(df2['time'] == time) & (df2['IMAGE Region Name'] == image_region) & (df2['NGFBFC'] == ngfbfc)]['value'].replace([np.inf, -np.inf], np.nan).values
            average_AreaHarvested_RegionCrops[n][i][j] = np.nan_to_num(ngfbfc_areaharves)

for n, time in enumerate(range(start_time_year+5, end_time_year, interval)):
    for i, ngfbfc in enumerate(df1['NGFBFC'].unique()):
        carbon_footprint = df1[(df1['time'] == time) & (df1['NGFBFC'] == ngfbfc)]['CF'].replace([np.inf, -np.inf], np.nan).values
        data1[n][i] = np.nan_to_num(carbon_footprint)

for i, ngfbfc in enumerate(df1['NGFBFC'].unique()):    
    for n, time in enumerate(range(start_time_year+5, end_time_year, interval)):
        result_average[i][n] = np.average(data1[n][i], weights=average_AreaHarvested_RegionCrops[n][i])

df_avg = pd.DataFrame(result_average, index=[df2['NGFBFC'].unique()], columns=list(range(start_time_year+5, end_time_year, interval)))
df_avg = df_avg.reset_index()
df_avg = df_avg.rename(columns={"level_0" : 'NGFBFC'})
df_avg_melt = pd.melt(
    df_avg, 
    id_vars=["NGFBFC"],  
    var_name="time",     
    value_name="value"  
)
df_avg = df_avg_melt.sort_values(by=['value', 'NGFBFC'], ascending=[False, False])

results = {}
for category in df_avg['NGFBFC'].unique():
    for time in range(start_time_year+5, end_time_year, interval):
        filtered_data = df_avg[(df_avg['NGFBFC'] == category) & (df_avg['time'] == time)]
        sorted_data = filtered_data.sort_values(by='value', ascending=False)
        top_5_data = sorted_data.head(3)
        if time not in results:
            results[time] = {}
        results[time][category] = top_5_data

df_avg = pd.concat([pd.concat(results[year].values()) for year in results.keys()])
df_avg = df_avg.sort_values(by=['time', 'value'], ascending=[True, False])
df_avg = ngfbfc_processing(df_avg)

produk_diulang = np.array([item for item in df_avg[df_avg['time'] == 1980]['NGFBFC'].to_list() for _ in range(1)])

df_footprint_region = region_processing(df1)
df_footprint_region_table = pd.pivot_table(data=df_footprint_region, index=['NGFBFC', 'time'], columns=['IMAGE Region Name'])
df_footprint_region_index = df_footprint_region_table.stack(level=0, future_stack=True)
df_footprint_region_index = df_footprint_region_index.reset_index()
df_footprint_region_index_drop = df_footprint_region_index.drop(columns=['level_2'])
df_footprint_region_index_drop.index = df_footprint_region_index_drop.columns['NGFBFC'].values

average1_melt = df_footprint_region_index_drop.melt(id_vars=['NGFBFC', 'time'], value_name='value') 
average1_melt = average1_melt[~((average1_melt['IMAGE Region Name'] == "Canada"))]
average1_melt = average1_melt[~((average1_melt['IMAGE Region Name'] == "Central America"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "tropical cereals") & (average1_melt['IMAGE Region Name'] == "Japan"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "palm oil") & (average1_melt['IMAGE Region Name'] == "South Africa"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "palm oil") & (average1_melt['IMAGE Region Name'] == "Rest of Southern Africa"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "palm oil") & (average1_melt['IMAGE Region Name'] == "USA"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "non food, luxury, spices") & (average1_melt['IMAGE Region Name'] == "Russia region"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "soybeans") & (average1_melt['IMAGE Region Name'] == "Southeast Asia"))]

average1_melt.index = average1_melt.columns[:,'NGFBFC'].values
average1_filtered = average1_melt[average1_melt['value'] < 20]
average1_new = average1_filtered.sort_values(by=['value', 'NGFBFC'], ascending=[True, True])

results = {}
for country in average1_new['IMAGE Region Name'].unique():
    for category in average1_new['NGFBFC'].unique():
        for time in range(start_time_year+5, end_time_year, interval):
            filtered_data = average1_new[(average1_new['NGFBFC'] == category) & (average1_new['time'] == time)]
            sorted_data = filtered_data.sort_values(by='value', ascending=False)
            top_5_data = sorted_data.head(3)
            if time not in results:
                results[time] = {}
            results[time][category] = top_5_data

df_footprint_df = pd.concat([pd.concat(results[year].values()) for year in results.keys()])
df_footprint_df_filtered = df_footprint_df[df_footprint_df['value'] < 20]

for time in range(start_time_year+5, end_time_year, interval):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlabel('Crop Footprint based on total area ($ton~CO_2/~hectare$)')
    ax.set_ylabel('Crop Categories')

    plt.title(f"{time}")
    
    produk_diulang = np.array([item for item in df_avg[df_avg['time'] == time]['NGFBFC'].to_list() for _ in range(1)])

    sns.scatterplot(x="value", y="NGFBFC", color='#686D76', sizes=(100, 300), alpha=.3, data=average1_filtered[average1_filtered['time'] == time].loc[produk_diulang])
    sns.scatterplot(x="value", y="NGFBFC", hue='IMAGE Region Name', sizes=(100, 300), alpha=.8, palette=palettes, data=df_footprint_df_filtered[df_footprint_df_filtered['time'] == time])

    ax.legend(bbox_to_anchor=(1.01, 1), ncol=1, loc='upper left')
    ax.plot(df_avg[df_avg['time'] == time].sort_values(by='value', ascending=False)['value'].to_numpy(), df_avg[df_avg['time'] == time].sort_values(by='value', ascending=False)['NGFBFC'].to_numpy(), "D", color='red', label='Weighted Average')
    
    legend_handles, legend_labels = ax.get_legend_handles_labels()
    weighted_average_index = legend_labels.index('Weighted Average')
    weighted_average_handle = legend_handles.pop(weighted_average_index)
    weighted_average_label = legend_labels.pop(weighted_average_index)
    
    legend_handles.append(weighted_average_handle)
    legend_labels.append(weighted_average_label)

    ax.legend(handles=legend_handles, labels=legend_labels, bbox_to_anchor=(1.01, 1), ncol=1, loc='upper left')
    ax.xaxis.set_major_locator(MultipleLocator(5))
    
    plt.show()
    plt.savefig(f'/VIZ/CF_multiyears_{time}.png')

df1 = CF_emisi_total_AH.copy()
df2 = df_GFRAC_Area_selected.copy()

df1.loc[df1['CF'] > 50, 'CF'] = 0
df1 = ngfbfc_processing(df1)
df2 = ngfbfc_processing(df2)

data1 = np.zeros((len(df1['NGFBFC'].unique()), len(df2['IMAGE Region Name'].unique())), dtype='float32')
average_AreaHarvested_RegionCrops = np.zeros((len(df1['NGFBFC'].unique()), len(df2['IMAGE Region Name'].unique())), dtype='float32')
result_average = np.zeros((len(df1['NGFBFC'].unique())), dtype='float32')

for i, ngfbfc in enumerate(df1['NGFBFC'].unique()):
    for n, image_region in enumerate(df2['IMAGE Region Name'].unique()):
        ngfbfc_areaharves = df2[(df2['IMAGE Region Name'] == image_region) & (df2['NGFBFC'] == ngfbfc) & (df2['value'] > 0)]['value'].replace([np.inf, -np.inf], np.nan).mean()
        average_AreaHarvested_RegionCrops[i][n] = np.nan_to_num(ngfbfc_areaharves)

for i, ngfbfc in enumerate(df1['NGFBFC'].unique()):
    carbon_footprint = df1[df1['NGFBFC'] == ngfbfc]['CF'].replace([np.inf, -np.inf], np.nan).mean()
    data1[i] = np.nan_to_num(carbon_footprint)

for x in range(len(df1['NGFBFC'].unique())):
    result_average[x] = np.average(data1[x], weights=average_AreaHarvested_RegionCrops[x])

df_avg = pd.DataFrame(result_average, index=df2['NGFBFC'].unique(), columns=['value'])
df_avg = df_avg.reset_index()
df_avg = df_avg.rename(columns={"index":'NGFBFC'})
df_avg = df_avg.sort_values(by=['value'], ascending=[False])

produk_diulang = np.array([item for item in df_avg['NGFBFC'].to_list() for _ in range(1)])

df_footprint_region = region_processing(df1)
df_footprint_region_table = pd.pivot_table(data=df_footprint_region, index=['NGFBFC', 'time'], columns=['IMAGE Region Name'])
df_footprint_region_index = df_footprint_region_table.stack(level=0, future_stack=True)
df_footprint_region_index = df_footprint_region_index.reset_index()
df_footprint_region_index_drop = df_footprint_region_index.drop(columns=['level_2'])
df_footprint_region_index_drop = df_footprint_region_index_drop.drop(columns=['time'])
df_footprint_region_index_drop.index = df_footprint_region_index_drop.loc[:, 'NGFBFC'].values

average1_melt = df_footprint_region_index_drop.melt(id_vars=['NGFBFC'], value_name='value') 
average1_melt = average1_melt[~((average1_melt['IMAGE Region Name'] == "Canada"))]
average1_melt = average1_melt[~((average1_melt['IMAGE Region Name'] == "Central America"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "tropical cereals") & (average1_melt['IMAGE Region Name'] == "Japan"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "palm oil") & (average1_melt['IMAGE Region Name'] == "South Africa"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "palm oil") & (average1_melt['IMAGE Region Name'] == "Rest of Southern Africa"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "palm oil") & (average1_melt['IMAGE Region Name'] == "USA"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "non food, luxury, spices") & (average1_melt['IMAGE Region Name'] == "Russia region"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "soybeans") & (average1_melt['IMAGE Region Name'] == "Southeast Asia"))]

average = average1_melt.groupby(['IMAGE Region Name', 'NGFBFC'])['value'].mean().to_frame('Mean')
average1 = average.reset_index()
average1.index = average1.columns['NGFBFC'].values
average1_filtered = average1[average1['Mean'] < 20]
average1_new = average1_filtered.sort_values(by=['Mean', 'NGFBFC'], ascending=[True, True])

results = {}
for country in average1_new['IMAGE Region Name'].unique():
    for category in average1_new['NGFBFC'].unique():
        filtered_data = average1_new[average1_new['NGFBFC'] == category]
        sorted_data = filtered_data.sort_values(by='Mean', ascending=False)
        top_5_data = sorted_data.head(3)
        results[category] = top_5_data

df_footprint_df = pd.concat([pd.concat(results.values())])
df_footprint_df_filtered = df_footprint_df[df_footprint_df['Mean'] < 20]

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlabel('Crop Footprint based on total area ($ton~CO_2/~hectare$)')
ax.set_ylabel('Crop Categories')

sns.scatterplot(x="Mean", y="NGFBFC", color='#686D76', sizes=(100, 300), alpha=.3, data=average1_filtered.loc[produk_diulang])
sns.scatterplot(x="Mean", y="NGFBFC", hue='IMAGE Region Name', sizes=(100, 300), alpha=.8, palette=palettes, data=df_footprint_df_filtered)

ax.legend(bbox_to_anchor=(1.01, 1), ncol=1, loc='upper left')
ax.plot(df_avg['value'].to_numpy(), df_avg['NGFBFC'].to_numpy(), "D", color='red', label='Weighted Average')

legend_handles, legend_labels = ax.get_legend_handles_labels()
weighted_average_index = legend_labels.index('Weighted Average')
weighted_average_handle = legend_handles.pop(weighted_average_index)
weighted_average_label = legend_labels.pop(weighted_average_index)

legend_handles.append(weighted_average_handle)
legend_labels.append(weighted_average_label)

ax.legend(handles=legend_handles, labels=legend_labels, bbox_to_anchor=(1.01, 1), ncol=1, loc='upper left')
ax.xaxis.set_major_locator(MultipleLocator(5))

plt.show()
plt.savefig(f'/VIZ/CF_average_{start_time_year+5}-{end_time_year}.png')

In [ ]:
emisi_total_copy = all_sources.copy()
emisi_total_copy = ngfbfc_processing(emisi_total_copy)
emisi_total_copy = emisi_total_copy[emisi_total_copy['NGFBFC'] != "pasture"]
emisi_total_copy = emisi_total_copy.groupby(['IMAGE Region Name', 'NGFBFC']).sum()
emisi_total_copy = emisi_total_copy.reset_index()
emisi_total_copy = emisi_total_copy.drop(columns=['emissions'])

emisi_total_melted = pd.melt(emisi_total_copy, id_vars=['IMAGE Region Name', 'NGFBFC'], var_name='time', value_name='value')
emisi_total_melted = emisi_total_melted.groupby(['IMAGE Region Name', 'time', 'NGFBFC']).sum()
emisi_total_melted = emisi_total_melted.reset_index()
emisi_total_melted['value'] = emisi_total_melted['value'] * 1E9

fao_prod_copy = faostat_prod_2a.copy()
fao_prod_copy = ngfbfc_processing(fao_prod_copy)
fao_prod_copy = fao_prod_copy[fao_prod_copy['time'] >= start_time_year+5]

df_intensity_region = fao_prod_copy.copy()
for region, time, ngfbfc in zip(df_intensity_region["IMAGE Region Name"].to_list(), df_intensity_region["time"].to_list(), df_intensity_region["NGFBFC"].to_list()):
    try:
        data1 = fao_prod_copy[(fao_prod_copy['IMAGE Region Name'] == region) & (fao_prod_copy['time'] == time) & (fao_prod_copy['NGFBFC'] == ngfbfc)]['FAO Production'].to_numpy()
        data2 = emisi_total_melted[(emisi_total_melted['IMAGE Region Name'] == region) & (emisi_total_melted['time'] == time) & (emisi_total_melted['NGFBFC'] == ngfbfc)]['value'].to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        mask = df_intensity_region[(df_intensity_region['IMAGE Region Name'] == region) & (df_intensity_region['time'] == time) & (df_intensity_region['NGFBFC'] == ngfbfc)].index
        df_intensity_region.loc[mask, 'Carbon Intensity'] =  data2[0] / data1[0]
    
    except (KeyError, ZeroDivisionError) as e:
        print(f"Error: {e} for region {region}")

df_intensity_region = df_intensity_region.drop(columns=['FAO Production'])
df_intensity_region['Carbon Intensity'] = df_intensity_region['Carbon Intensity'].replace([np.inf, -np.inf], np.nan)
df_intensity_region['Carbon Intensity'] = df_intensity_region['Carbon Intensity'].fillna(0)

df1_copy = df_intensity_region.copy()
df2_copy = faostat_prod_2a.copy()

df1_copy.loc[df1_copy['Carbon Intensity'] > 50, 'Carbon Intensity'] = 0
df1_copy = ngfbfc_processing(df1_copy)
df2_copy = ngfbfc_processing(df2_copy)

df_ittertols = df_footprint_region.drop(columns=['CF'])
df_itertools_copy1 = df_ittertols.copy()
df_itertools_copy1["Carbon Intensity"] = 0.0
df1_copy = df_itertools_copy1.merge(df1_copy , how='left', left_on=['IMAGE Region Name', 'time', 'NGFBFC'], right_on=['IMAGE Region Name', 'time', 'NGFBFC'])
df1_copy = df1_copy.drop(columns=['Carbon Intensity_x']).rename(columns={"Carbon Intensity_y": "Carbon Intensity"})

df_itertools_copy2 = df_ittertols.copy()
df_itertools_copy2["FAO Production"] = 0.0
df2_copy = df_itertools_copy2.merge(df2_copy , how='left', left_on=['IMAGE Region Name', 'time', 'NGFBFC'], right_on=['IMAGE Region Name', 'time', 'NGFBFC'])
df2_copy = df2_copy.drop(columns=['FAO Production_x']).rename(columns={"FAO Production_y": "FAO Production"})

data1 = np.zeros((len(df1_copy['time'].unique()), len(df1_copy['NGFBFC'].unique()), len(df1_copy['IMAGE Region Name'].unique())), dtype='float32')
average_AreaHarvested_RegionCrops = np.zeros((len(df1_copy['time'].unique()), len(df1_copy['NGFBFC'].unique()), len(df1_copy['IMAGE Region Name'].unique())), dtype='float32')
result_average = np.zeros((len(df1_copy['NGFBFC'].unique()), len(df1_copy['time'].unique())), dtype='float32')
for n, time in enumerate(range(start_time_year+5, end_time_year, interval)):
    for i, ngfbfc in enumerate(df1_copy['NGFBFC'].unique()):
        for j, image_region in enumerate(df2_copy['IMAGE Region Name'].unique()):
            ngfbfc_areaharves = df2_copy[(df2_copy['time'] == time) & (df2_copy['IMAGE Region Name'] == image_region) & (df2_copy['NGFBFC'] == ngfbfc)]['FAO Production'].replace([np.inf, -np.inf], np.nan).values
            average_AreaHarvested_RegionCrops[n][i][j] = np.nan_to_num(ngfbfc_areaharves)

for n, time in enumerate(range(start_time_year+5, end_time_year, interval)):
    for i, ngfbfc in enumerate(df1_copy['NGFBFC'].unique()):
        carbon_footprint = df1_copy[(df1_copy['time'] == time) & (df1_copy['NGFBFC'] == ngfbfc)]['Carbon Intensity'].replace([np.inf, -np.inf], np.nan).values
        data1[n][i] = np.nan_to_num(carbon_footprint)

for i, ngfbfc in enumerate(df1_copy['NGFBFC'].unique()):   
    for n, time in enumerate(range(start_time_year+5, end_time_year, interval)):
        result_average[i][n] = np.average(data1[n][i], weights=average_AreaHarvested_RegionCrops[n][i])

df_avg = pd.DataFrame(result_average, index=[df2_copy['NGFBFC'].unique()], columns=list(range(start_time_year+5, end_time_year, interval)))
df_avg = df_avg.reset_index()
df_avg = df_avg.rename(columns={"level_0":'NGFBFC'})
df_avg_melt = pd.melt(
    df_avg, 
    id_vars=["NGFBFC"], 
    var_name="time",     
    value_name="value"   
)
df_avg = df_avg_melt.sort_values(by=['value', 'NGFBFC'], ascending=[False, False])

results = {}
for category in df_avg['NGFBFC'].unique():
    for time in range(start_time_year+5, end_time_year, interval):
        filtered_data = df_avg[(df_avg['NGFBFC'] == category) & (df_avg['time'] == time)]
        sorted_data = filtered_data.sort_values(by='value', ascending=False)
        top_5_data = sorted_data.head(3)
        if time not in results:
            results[time] = {}
        results[time][category] = top_5_data

df_avg = pd.concat([pd.concat(results[year].values()) for year in results.keys()])
df_avg = df_avg.sort_values(by=['time', 'value'], ascending=[True, False])

df_footprint_region = region_processing(df1_copy)
df_footprint_region_table = pd.pivot_table(data=df_footprint_region, index=['NGFBFC', 'time'], columns=['IMAGE Region Name'])
df_footprint_region_index = df_footprint_region_table.stack(level=0, future_stack=True)
df_footprint_region_index = df_footprint_region_index.reset_index()
df_footprint_region_index_drop = df_footprint_region_index.drop(columns=['level_2'])
df_footprint_region_index_drop.index = df_footprint_region_index_drop.columns['NGFBFC'].values
  
average1_melt = df_footprint_region_index_drop.melt(id_vars=['NGFBFC', 'time'], value_name='value') 
average1_melt = average1_melt[~((average1_melt['IMAGE Region Name'] == "Canada"))]
average1_melt = average1_melt[~((average1_melt['IMAGE Region Name'] == "Central America"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "tropical cereals") & (average1_melt['IMAGE Region Name'] == "Japan"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "palm oil") & (average1_melt['IMAGE Region Name'] == "South Africa"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "palm oil") & (average1_melt['IMAGE Region Name'] == "Rest of Southern Africa"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "palm oil") & (average1_melt['IMAGE Region Name'] == "USA"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "non food, luxury, spices") & (average1_melt['IMAGE Region Name'] == "Russia region"))]
average1_melt = average1_melt[~((average1_melt['NGFBFC'] == "soybeans") & (average1_melt['IMAGE Region Name'] == "Southeast Asia"))]

average1_melt.index = average1_melt.columns['NGFBFC'].values
average1_melt = average1_melt[average1_melt['value'] < 20]
average1_melt = average1_melt.sort_values(by=['value', 'NGFBFC'], ascending=[True, True])

results = {}
for country in average1_melt['IMAGE Region Name'].unique():
    for category in average1_melt['NGFBFC'].unique():
        for time in range(start_time_year+5, end_time_year, interval):
            filtered_data = average1_melt[(average1_melt['NGFBFC'] == category) & (average1_melt['time'] == time)]
            sorted_data = filtered_data.sort_values(by='value', ascending=False)
            top_5_data = sorted_data.head(3)
            if time not in results:
                results[time] = {}
            results[time][category] = top_5_data

df_footprint_df = pd.concat([pd.concat(results[year].values()) for year in results.keys()])
df_footprint_df_filtered = df_footprint_df[df_footprint_df['value'] < 20]

for time in range(start_time_year+5, end_time_year, interval):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlabel('Crop Footprint based on total area ($ton~CO_2/~hectare$)')
    ax.set_ylabel('Crop Categories')

    plt.title(f"{time}")
    
    produk_diulang = np.array([item for item in df_avg[df_avg['time'] == time]['NGFBFC'].to_list() for _ in range(1)])

    sns.scatterplot(x="value", y="NGFBFC", color='#686D76', sizes=(100, 300), alpha=.3, data=average1_filtered[average1_filtered['time'] == time].loc[produk_diulang])
    sns.scatterplot(x="value", y="NGFBFC", hue='IMAGE Region Name', sizes=(100, 300), alpha=.8, palette=palettes, data=df_footprint_df_filtered[df_footprint_df_filtered['time'] == time])

    ax.legend(bbox_to_anchor=(1.01, 1), ncol=1, loc='upper left')
    ax.plot(df_avg[df_avg['time'] == time].sort_values(by='value', ascending=False)['value'].to_numpy(), df_avg[df_avg['time'] == time].sort_values(by='value', ascending=False)['NGFBFC'].to_numpy(), "D", color='red', label='Weighted Average')
    
    legend_handles, legend_labels = ax.get_legend_handles_labels()
    weighted_average_index = legend_labels.index('Weighted Average')
    weighted_average_handle = legend_handles.pop(weighted_average_index)
    weighted_average_label = legend_labels.pop(weighted_average_index)
    
    legend_handles.append(weighted_average_handle)
    legend_labels.append(weighted_average_label)

    ax.legend(handles=legend_handles, labels=legend_labels, bbox_to_anchor=(1.01, 1), ncol=1, loc='upper left')
    ax.xaxis.set_major_locator(MultipleLocator(5))

    plt.show()
    plt.savefig(f'/VIZ/CI_multiyears_{time}.png')

df1_copy = df_intensity_region.copy()
df2_copy = faostat_prod_2a.copy()

df1_copy.loc[df1_copy['Carbon Intensity'] > 50, 'Carbon Intensity'] = 0
df1_copy= ngfbfc_processing(df1_copy)
df2_copy = ngfbfc_processing(df2_copy)

df2_copy = df2_copy[~((df2_copy['time'] <= 1990) & (df2_copy['IMAGE Region Name'] == "Russia"))]
df2_copy = df2_copy[df2_copy['time'] >= start_time_year+5]

data1 = np.zeros((len(df1_copy['NGFBFC'].unique()), len(df2_copy['IMAGE Region Name'].unique())), dtype='float32')
average_AreaHarvested_RegionCrops = np.zeros((len(df1_copy['NGFBFC'].unique()), len(df2_copy['IMAGE Region Name'].unique())), dtype='float32')
result_average = np.zeros((len(df1_copy['NGFBFC'].unique())), dtype='float32')
for i, ngfbfc in enumerate(df1_copy['NGFBFC'].unique()):
    for n, image_region in enumerate(df2_copy['IMAGE Region Name'].unique()):
        ngfbfc_areaharves = df2_copy[(df2_copy['IMAGE Region Name'] == image_region) & (df2_copy['NGFBFC'] == ngfbfc) & (df2_copy['FAO Production'] > 0)]['FAO Production'].replace([np.inf, -np.inf], np.nan).mean()
        average_AreaHarvested_RegionCrops[i][n] = np.nan_to_num(ngfbfc_areaharves)

for i, ngfbfc in enumerate(df1_copy['NGFBFC'].unique()):
    carbon_footprint = df1_copy[df1_copy['NGFBFC'] == ngfbfc]['Carbon Intensity'].replace([np.inf, -np.inf], np.nan).mean()
    data1[i] = np.nan_to_num(carbon_footprint)

for i, ngfbfc in enumerate(df1_copy['NGFBFC'].unique()):   
    result_average[i] = np.average(data1[i], weights=average_AreaHarvested_RegionCrops[i])
    
df_avg = pd.DataFrame(result_average, index=df2_copy['NGFBFC'].unique())
df_avg = df_avg.reset_index()
df_avg = df_avg.rename(columns={"index":'NGFBFC', 0:"value"})
df_avg = df_avg.sort_values(by=['value'], ascending=[False])

df_footprint_region = region_processing(df1_copy)
df_footprint_region = df_footprint_region[~((df_footprint_region['time'] <= 1990) & (df_footprint_region['IMAGE Region Name'] == "Russia"))] 

df_footprint_region_table = pd.pivot_table(data=df_footprint_region, index=['NGFBFC', 'time'], columns=['IMAGE Region Name'])
df_footprint_region_index = df_footprint_region_table.stack(level=0, future_stack=True)
df_footprint_region_index = df_footprint_region_index.reset_index()
df_footprint_region_index = df_footprint_region_index.drop(columns=['level_2'])
df_footprint_region_index.index = df_footprint_region_index['NGFBFC'].values

produk_diulang = np.array([item for item in df_avg['NGFBFC'].to_list() for _ in range(1)])
df_footprint_region_index = df_footprint_region_index.loc[produk_diulang]
df_footprint_region_index = df_footprint_region_index.drop(columns=['time'])

average_melt = df_footprint_region_index.melt(id_vars=['NGFBFC'], value_name='value') 
average_melt['value'] = average_melt['value'].fillna(0)

average_melt = average_melt.groupby(['IMAGE Region Name', 'NGFBFC'])['value'].apply(custom_mean).to_frame('Mean').reset_index()
average_melt.index = average_melt['NGFBFC'].values
average_melt = average_melt[average_melt['Mean'] < 25]

average1_new = average.sort_values(by=['Mean', 'NGFBFC'], ascending=[True, True])
average1_new = average1_new[~((average1_new['IMAGE Region Name'] == "Canada"))]
average1_new = average1_new[~((average1_new['IMAGE Region Name'] == "Central America"))]
average1_new = average1_new[~((average1_new['IMAGE Region Name'] == "Japan"))]
average1_new = average1_new[~((average1_new['IMAGE Region Name'] == "Central Europe"))]
average1_new = average1_new[~((average1_new['NGFBFC'] == "palm oil") & (average1_new['IMAGE Region Name'] == "South Africa"))]
average1_new = average1_new[~((average1_new['NGFBFC'] == "palm oil") & (average1_new['IMAGE Region Name'] == "Rest of Southern Africa"))]
average1_new = average1_new[~((average1_new['NGFBFC'] == "palm oil") & (average1_new['IMAGE Region Name'] == "USA"))]
average1_new = average1_new[~((average1_new['NGFBFC'] == "non food, luxury, spices") & (average1_new['IMAGE Region Name'] == "Russia region"))]
average1_new = average1_new[~((average1_new['NGFBFC'] == "soybeans") & (average1_new['IMAGE Region Name'] == "Southeast Asia"))]
average1_new = average1_new[~((average1_new['NGFBFC'] == "soybeans") & (average1_new['IMAGE Region Name'] == "Indonesia region"))]
average1_new = average1_new[~((average1_new['NGFBFC'] == "plant based fibres") & (average1_new['IMAGE Region Name'] == "Ukraine region"))]

results = {}
for country in average1_new['IMAGE Region Name'].unique():
    for category in average1_new['NGFBFC'].unique():
        filtered_data = average1_new[average1_new['NGFBFC'] == category]
        sorted_data = filtered_data.sort_values(by='Mean', ascending=False)
        top_5_data = sorted_data.head(3)
        results[category] = top_5_data

df_footprint_df = pd.concat([pd.concat(results.values())])
df_footprint_df = df_footprint_df[df_footprint_df['Mean'] < 25]

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlabel('Crop Emission Intensity ($ton~CO_2-eq/~tonne~crop$)')
ax.set_ylabel('Crop Categories')

sns.scatterplot(x="Mean", y="NGFBFC", color='#686D76', sizes=(100, 300), alpha=.3, data=average1_new.loc[produk_diulang])
sns.scatterplot(x="Mean", y="NGFBFC", hue='IMAGE Region Name', sizes=(100, 300), alpha=.8, palette=palettes, data=df_footprint_df)

ax.legend(bbox_to_anchor=(1.01, 1), ncol=1, loc='upper left')
ax.plot(df_avg['value'].to_numpy(), df_avg['NGFBFC'].to_numpy(), "D", color='red', label='Weighted Average')

legend_handles, legend_labels = ax.get_legend_handles_labels()
weighted_average_index = legend_labels.index('Weighted Average')
weighted_average_handle = legend_handles.pop(weighted_average_index)
weighted_average_label = legend_labels.pop(weighted_average_index)

legend_handles.append(weighted_average_handle)
legend_labels.append(weighted_average_label)

ax.legend(handles=legend_handles, labels=legend_labels, bbox_to_anchor=(1.01, 1), ncol=1, loc='upper left')

plt.show()
plt.savefig(f'/VIZ/CI_average_{start_time_year+5}-{end_time_year}.png')

In [ ]:

columns_to_multiply = [int(year) for year in range(start_time_year, end_time_year+1, 5)]

df_LUC = all_sources[(all_sources['emissions'] == 'LUC_Agri') | (all_sources['emissions'] == 'agri2agri') | (all_sources['emissions'] == 'GEN2OLC')]
df_LUC = df_LUC.groupby(['IMAGE Region Name', 'type']).sum()
df_LUC = df_LUC.reset_index()
df_LUC = df_LUC.drop(columns=['emissions'])
df_LUC.loc[:, columns_to_multiply] = df_LUC.loc[:, columns_to_multiply] * 5
df_LUC_melt = df_LUC.melt(id_vars=['IMAGE Region Name', 'type'], var_name='time', value_name='value')
df_LUC_melt_droptime = df_LUC_melt.drop(columns=['time'])
df_LUC_melt_droptime = df_LUC_melt_droptime.groupby(['IMAGE Region Name', 'type']).sum()
df_LUC_index = df_LUC_melt_droptime.reset_index()
df_LUC_index.loc[:, 'emissions'] = "LUC"

df_AGRI = all_sources[~(all_sources['emissions'] == 'LUC_Agri') | ~(all_sources['emissions'] == 'agri2agri') | ~(all_sources['emissions'] == 'GEN2OLC')]
df_AGRI = df_AGRI.groupby(['IMAGE Region Name', 'type']).sum()
df_AGRI = df_AGRI.reset_index()
df_AGRI = df_AGRI.drop(columns=['emissions'])
df_AGRI.loc[:, columns_to_multiply] = df_AGRI.loc[:, columns_to_multiply] * 5
df_AGRI_melt = df_AGRI.melt(id_vars=['IMAGE Region Name', 'type'], var_name='time', value_name='value')
df_agrim_melt_droptime = df_AGRI_melt.drop(columns=['time'])
df_agrim_melt_droptime = df_agrim_melt_droptime.groupby(['IMAGE Region Name', 'type']).sum()
df_agrim_index = df_agrim_melt_droptime.reset_index()
df_agrim_index.loc[:, 'emissions'] = "AGRI"

data_2kategori = pd.concat([df_agrim_index, df_LUC_index], ignore_index=True)
data_2kategori_drop = data_2kategori.drop(columns=['emissions'])
data_2kategori_sum = data_2kategori_drop.groupby(['IMAGE Region Name', 'type']).sum()
data_2kategori_sum_index = data_2kategori_sum.reset_index()
data_2kategori_sum_index['type'] = data_2kategori_sum_index['type'].replace('grass', 'pasture').replace('oil & palm fruit', 'palm oil')
data_2kategori_sum_index['type'] = data_2kategori_sum_index['type'].replace('other temperate cereals', 'temperate cereals')
data_2kategori_sum_index['IMAGE Map'] = data_2kategori_sum_index['IMAGE Region Name']
data_2kategori_sum_index = region_processing(data_2kategori_sum_index)
data_2kategori_sum_index['IMAGE Classification Region'] = data_2kategori_sum_index['IMAGE Region Name'] + "-" + data_2kategori_sum_index['type']
data_2kategori_sum_index = data_2kategori_sum_index[data_2kategori_sum_index['IMAGE Classification Region'] != 'indonesia region-temperate cereals'] #drop karena tidak ada produksi di FAO

df_top3 = get_top_n(data_2kategori_sum_index, 'type', 'value', 2)

region_code_map = country_code.set_index('IMAGE Region Name')['IMAGE Region Code'].to_dict()
df_top3['IMAGE Map'] = df_top3['IMAGE Map'].map(region_code_map)
df_top3 = df_top3[df_top3['type'] != 'pasture']
df_top3 = df_top3.drop(columns=['IMAGE Region Name'])

columns2 = ['plant based fibres', 'other non-food & luxury & spices', 'vegetables & fruits', 'palm oil',
            'sugar crops', 'tropical roots & tubers', 'temperate roots & tubers', 'tropical oil crops',
            'temperate oil crops', 'soybeans', 'pulses', 'temperate cereals', 'tropical cereals', 'maize', 'rice',
            'wheat', 'pasture']

colors2 = ['#4D869C', '#7AB2B2', '#CDE8E5', '#FC4100', '#FFC55A', '#8E3E63', '#D2649A', '#03AED2', 
'#68D2E8', '#5F6F52', '#A9B388', '#FC819E', '#F7418F', '#FFF455', '#FFEFEF', '#F7C566', '#ACE1AF']

color_mapping = dict(zip(columns2, colors2))
def get_color(plant_type):
    return color_mapping.get(plant_type, 'unknown')

df_top3['color'] = df_top3['type'].apply(get_color)
df_pallete = df_top3.copy()

yield_raw = faostat_prod_2a.copy()
for region, time, ngfbfc in zip(yield_raw["IMAGE Region Name"].to_list(), yield_raw["time"].to_list(), yield_raw["NGFBFC"].to_list()):
    try:
        data1 = faostat_area_harvested[(faostat_area_harvested['IMAGE Region Name'] == region) & (faostat_area_harvested['time'] == time) & (faostat_area_harvested['NGFBFC'] == ngfbfc)]['Fao Area Harvested'].to_numpy()
        data2 = faostat_prod_2a[(faostat_prod_2a['IMAGE Region Name'] == region) & (faostat_prod_2a['time'] == time) & (faostat_prod_2a['NGFBFC'] == ngfbfc)]['FAO Production'].to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        mask = yield_raw[(yield_raw['IMAGE Region Name'] == region) & (yield_raw['time'] == time) & (yield_raw['NGFBFC'] == ngfbfc)].index
        yield_raw.loc[mask, 'value'] =  data2[0] / data1[0]

    except (KeyError, ZeroDivisionError) as e:
        print(f"Error: {e} for region {region}")

yield_raw_1 = yield_raw.copy()
yield_raw_1 = yield_raw_1.drop(columns=['FAO Production'])
yield_raw_1 = yield_raw_1[yield_raw_1['time'] > start_time_year]

yield_raw_1['NGFBFC'] = yield_raw_1['NGFBFC'].str.lower()
yield_raw_1['NGFBFC'] = yield_raw_1['NGFBFC'].str.replace("tropical roots and tubers", "tropical roots & tubers")
yield_raw_1['NGFBFC'] = yield_raw_1['NGFBFC'].str.replace("vegetables or fruits", "vegetables & fruits")
yield_raw_1['NGFBFC'] = yield_raw_1['NGFBFC'].str.replace("other temperate cereals", "temperate cereals")
yield_raw_1['NGFBFC'] = yield_raw_1['NGFBFC'].str.replace("temperate roots and tubers", "temperate roots & tubers")
yield_raw_1['NGFBFC'] = yield_raw_1['NGFBFC'].str.replace("plant-based fibers", "plant based fibres")
yield_raw_1['NGFBFC'] = yield_raw_1['NGFBFC'].str.replace("oil & palm fruit", "palm oil")

yield_raw_1 = region_processing(yield_raw_1)

yield_raw_copy = yield_raw_1.copy()
yield_raw_copy['IMAGE Classification Region'] = yield_raw_copy['IMAGE Region Name'] + "-" + yield_raw_copy['NGFBFC']
yield_raw_copy.rename(columns={'time': 'Year'}, inplace=True)
yield_raw_copy = yield_raw_copy.drop(columns=['IMAGE Region Name', 'NGFBFC'])
yield_raw_copy['value'] = yield_raw_copy['value'].fillna(0.0)
yield_FAO = yield_raw_copy[['Year', 'IMAGE Classification Region', 'value']]
yield_FAO  = yield_FAO[yield_FAO['IMAGE Classification Region'].isin(df_pallete['IMAGE Classification Region'])]

yield_FAO_average = yield_FAO.groupby(['IMAGE Classification Region'])['value'].apply(custom_mean).to_frame('value')
yield_FAO_average = yield_FAO_average.reset_index()
yield_FAO_average = yield_FAO_average[['IMAGE Classification Region', 'value']]
df2_array_selected = yield_FAO_average['value'].to_numpy()

carbon_footprint.rename(columns={'NGFBFC': 'type'}, inplace=True)
carbon_footprint['type'] = carbon_footprint['type'].str.replace("non food, luxury, spices", "other non-food & luxury & spices").replace("oil, palm fruit", "palm oil")
carbon_footprint = carbon_footprint[~carbon_footprint['type'].str.contains("pasture")]
carbon_footprint['IMAGE Region Name'] = carbon_footprint['IMAGE Region Name'].str.lower()
carbon_footprint['IMAGE Classification Region'] = carbon_footprint['IMAGE Region Name'] + "-" + carbon_footprint['type']
carbon_footprint_group = carbon_footprint.groupby(["IMAGE Classification Region"]).sum()
carbon_footprint_group = carbon_footprint_group.reset_index()
carbon_footprint_group.rename(columns={'Mean': 'average'}, inplace=True)

carbon_footprint_group['average'] = carbon_footprint_group['average'].fillna(0)
carbon_footprint_group = carbon_footprint_group[['IMAGE Classification Region', 'average']]
carbon_footprint_array = carbon_footprint_group['average'].to_numpy()
carbon_footprint_group_sorted = carbon_footprint_group.sort_values(by='average', ascending=False)
carbon_footprint_  = carbon_footprint_group[carbon_footprint_group['IMAGE Classification Region'].isin(df_pallete['IMAGE Classification Region'])]
carbon_footprint_array_selected = carbon_footprint_['average'].to_numpy()
carbon_footprint_ = pd.merge(left=carbon_footprint_, right=df_pallete, left_on="IMAGE Classification Region", right_on="IMAGE Classification Region", how="left")

crops = [x for x in carbon_footprint_group['IMAGE Classification Region'].unique()]

combinations = list(itertools.product(columns_to_multiply, crops))
df_new = pd.DataFrame(combinations, columns=['Year', 'IMAGE Classification Region'])
df_new["value"] = 0.0
df_new_copy2 = df_new.copy()
df_new_copy2 = df_new_copy2.merge(yield_FAO, how='left', left_on=['Year', 'IMAGE Classification Region'], right_on=['Year', 'IMAGE Classification Region'])
df_new_copy2['value'] = df_new_copy2['value_y'].fillna(0.0)
df_new_copy2 = df_new_copy2.drop(columns=['value_y', 'value_x'])
df_new_copy2 = pd.pivot_table(data=df_new_copy2, columns="Year", index=["IMAGE Classification Region"], values="value").reset_index()
df_new_copy2['average'] = df_new_copy2.iloc[:, 1:].mean(axis=1)
df_new_copy2['average'] = df_new_copy2['average'].fillna(0)
df_new_copy2 = df_new_copy2[['IMAGE Classification Region', 'average']]
df_2_sort = df_new_copy2.sort_values(by='average', ascending=False)
df2_array = df_new_copy2['average'].to_numpy()
df2_isin = df_new_copy2[df_new_copy2['IMAGE Classification Region'].isin(df_pallete['IMAGE Classification Region'])]
df2_array_selected = df2_isin['average'].to_numpy()

df2_colour = pd.merge(left=df2_isin, right=df_pallete, left_on="IMAGE Classification Region", right_on="IMAGE Classification Region", how="right")
df2_colour = df2_colour.sort_values(by='IMAGE Classification Region', ascending=True)

production_raw = faostat_prod_2a.copy()
production_raw = region_processing(production_raw)
production_raw['NGFBFC'] = production_raw['NGFBFC'].str.lower()
production_raw['NGFBFC'] = production_raw['NGFBFC'].str.replace('oil & palm fruit', 'palm oil')
production_raw['NGFBFC'] = production_raw['NGFBFC'].str.replace('other non-food & luxury & spices', 'other non-food & luxury & spices')
production_raw['NGFBFC'] = production_raw['NGFBFC'].str.replace('other temperate cereals', 'temperate cereals')
production_raw['IMAGE Classification Region'] = production_raw['IMAGE Region Name'] + "-" + production_raw['NGFBFC']
production_raw = production_raw.drop(columns=['IMAGE Region Name', 'NGFBFC'])

production_copy = production_raw.copy()
production_copy = production_copy.groupby(['IMAGE Classification Region'])['FAO Production'].apply(custom_mean).to_frame('average')
production_copy = production_copy.reset_index()

df_norm = production_copy.copy()
df_norm = df_norm[['IMAGE Classification Region', 'average']]
df_norm_selected = df_norm[df_norm['IMAGE Classification Region'].isin(df_pallete['IMAGE Classification Region'])]
df_norm_selected_test = df_norm_selected['average'].to_numpy()/1e7

df_agrim_melt_multiyear = df_AGRI_melt[((df_AGRI_melt['time'] > 1990))]
df_agrim_melt_multiyear = df_agrim_melt_multiyear.drop(columns=['time'])
df_agrim_groupby_multiyear = df_agrim_melt_multiyear.groupby(['IMAGE Region Name', 'type']).sum()
df_agrim_index_multiyear = df_agrim_groupby_multiyear.reset_index()
df_agrim_index_multiyear.loc[:, 'emissions'] = "AGRI"

df_LUC_melt_multiyear = df_LUC_melt[((df_LUC_melt['time'] > 1990))]
df_LUC_melt_multiyear = df_LUC_melt_multiyear.drop(columns=['time'])
df_LUC_groupby_multiyear = df_LUC_melt_multiyear.groupby(['IMAGE Region Name', 'type']).sum()
df_LUC_index_multiyear = df_LUC_groupby_multiyear.reset_index()
df_LUC_index_multiyear.loc[:, 'emissions'] = "LUC"

data_2kategori = pd.concat([df_agrim_index_multiyear, df_LUC_index_multiyear], ignore_index=True)

five_top_data_multiyear = data_2kategori[
    (data_2kategori['IMAGE Region Name'] == "Brazil") & (data_2kategori['type'] == "soybeans") 
  | (data_2kategori['IMAGE Region Name'] == "Indonesia") & (data_2kategori['type'] == "oil & palm fruit") 
  | (data_2kategori['IMAGE Region Name'] == "SE. Asia") & (data_2kategori['type'] == "rice") 
  | (data_2kategori['IMAGE Region Name'] == "W. Africa") & (data_2kategori['type'] == "tropical cereals") 
                                      
]
total_emissions = five_top_data_multiyear.groupby(['IMAGE Region Name', 'type'])['value'].sum()
total_emissions = total_emissions.reset_index()
total_emissions.columns = ['IMAGE Region Name', 'type', 'total_akumulatif']
five_top_data_multiyear = five_top_data_multiyear.merge(total_emissions, on=['IMAGE Region Name', 'type'])
five_top_data_multiyear['percentage'] = (five_top_data_multiyear['value'] / five_top_data_multiyear['total_akumulatif']) * 100
pivot_df = five_top_data_multiyear.pivot_table(index=['IMAGE Region Name', 'type'], columns='emissions', values='percentage', fill_value=0)
bar_width = 0.6 

fig = plt.figure(figsize=(15, 10))
gs = gridspec.GridSpec(3, 3, figure=fig, hspace=0.001, wspace=0.01, width_ratios=[7,0.5,2.5], height_ratios=[5,2,3])

countour_plot = fig.add_subplot(gs[:, :1])

crop_footprint = carbon_footprint_array
crop_footprint_selected = carbon_footprint_array_selected

yields = df2_array.flatten()
yields_selected = df2_array_selected.flatten()

land_effeciency = 1 / yields
land_effeciency_selected = 1 / yields_selected

n_points = 100
carbon_footprint_contour = np.linspace(0, np.max(crop_footprint_selected) + 1, n_points)
df2_contour = np.linspace(0, np.max(land_effeciency_selected) + 0.1, n_points)

X, Y = np.meshgrid(carbon_footprint_contour, df2_contour)
Z = X * Y 

class_ranges = [0.1, 0.2, 0.5, 1, 2, 4, 6, 8, 11, 17, 30, 40]

base_cmap = plt.cm.Greys
colors = base_cmap(np.linspace(0.0, 0.6, base_cmap.N))  # Slice to avoid pure white or black
custom_cmap = LinearSegmentedColormap.from_list("custom_greys", colors)

norm = BoundaryNorm(class_ranges, ncolors=custom_cmap.N, clip=True)

contour = countour_plot.contourf(X, Y, Z, levels=class_ranges, cmap=custom_cmap, norm=norm, edgecolors='Black')

countour_plot.set_xlabel('Crop Footprint (tCO₂-eq/ ha)')
countour_plot.set_ylabel('Land intensity for crop production (ha/ton)')
countour_plot.axis('on')

for x, data in enumerate(zip(df2_colour['IMAGE Classification Region'].to_list(), df2_colour['color'].to_list(), df2_colour['IMAGE Map'].to_list())):
    sizes = df_norm_selected_test[x] * 60
    countour_plot.scatter(crop_footprint_selected[x], land_effeciency_selected[x], color=data[1], edgecolors='grey', label=data[0], s=sizes)
    
    text = countour_plot.text(crop_footprint_selected[x], land_effeciency_selected[x] + 0.02, data[2], fontsize=8, color='black')
    text.set_path_effects([path_effects.withStroke(linewidth=2, foreground="white")])

legend_ax = fig.add_subplot(gs[0, 2])
legend_ax.axis('off') 
colors = ['#4D869C', '#7AB2B2', '#CDE8E5', '#FC4100', '#FFC55A', '#8E3E63', '#D2649A', '#03AED2', 
           '#68D2E8', '#5F6F52', '#A9B388', '#FC819E', '#F7418F', '#FFF455', '#FFEFEF', '#F7C566']

legend_handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markeredgecolor='grey',markersize=10) for color in colors]
columns2 = ['plant based fibres','non food, luxury, spices','vegetables & fruits','palm oil',
            'sugar crops','tropical roots & tubers','temperate roots & tubers','tropical oil crops',
            'temperate oil crops','soybeans','pulses','temperate cereals','tropical cereals','maize','rice', 'wheat']

legend_ax.legend(legend_handles, columns2, loc='lower left', bbox_to_anchor=(-0.10,0.1), ncol=1)

size_ax = fig.add_subplot(gs[1,2]) 
size_labels = ['5 Mton', ' 67 Mton','      370 Mton']
size_values = [30, 404.84181766, 2248.3112172]
for size, label in zip(size_values, size_labels):
    size_ax.scatter([], [], s=size, facecolor='white', edgecolor='grey', linewidth=1.5, label=label)  # Empty scatter for legend
size_ax.legend(loc='lower left', bbox_to_anchor=(-0.15,0.5), ncol=3, handletextpad=0, labelspacing=0, frameon=False)
size_ax.axis('off') 

ax_barchart = fig.add_subplot(gs[2, 2])
custom_labels = [
    "BRA\nSoybeans",
    "INDO\nPalm Oil",
    "SEA\nRice",
    "WAF\nTropCereals"]

pivot_df.plot(kind='bar', stacked=True, ax=ax_barchart, color=['green', 'blue',], legend=False, width=0.4)

ax_barchart.yaxis.set_major_locator(plticker.MultipleLocator(25))
ax_barchart.set_xticks(range(len(custom_labels)))
ax_barchart.set_xticklabels(custom_labels, rotation=0)
ax_barchart.set_ylabel('(%)', fontsize=10)
ax_barchart.grid(axis='y', linestyle='--', alpha=0.7)
ax_barchart.tick_params(axis='y', labelsize=10)
ax_barchart.set_xlabel("")

bar_legend = ax_barchart.legend( ['MGMT', 'LUC'],
    bbox_to_anchor=(0.5, 1),  # Centered below the plot
    loc='lower center', 
    ncol=2,  # Two columns
    title_fontsize=12,
    fontsize=10
)

plt.tight_layout()
plt.show()
plt.savefig(f'/VIZ/Scatterplot_trade-off_{start_time_year+5}-{end_time_year}.png')